In [ ]:
def calculate_update_ratio_metrics(grad_norm, weight_norm):
    """
    📊 Вычисляет метрики update ratio для мониторинга эффективности обучения
    
    Update ratio = grad_norm / (weight_norm + 1e-8) показывает относительную "силу" обновления:
    - < 1e-5: микроскопические обновления (возможно, слишком низкий LR)
    - 1e-4 - 1e-3: оптимальный диапазон
    - > 1e-3: слишком большие обновления (возможно, слишком высокий LR)
    
    Args:
        grad_norm (float): Норма градиента
        weight_norm (float): L2-норма весов модели
        
    Returns:
        dict: Словарь с метриками update ratio и их интерпретацией
    """
    update_ratio = grad_norm / (weight_norm + 1e-8)
    
    # Классифицируем update ratio для удобства анализа
    if update_ratio < 1e-5:
        status = "microscopic"  # 🔴 слишком маленькие обновления
        numeric_status = 1.0
        emoji = "🔴"
    elif update_ratio <= 1e-4:
        status = "small"  # 🟡 маленькие, но допустимые
        numeric_status = 2.0
        emoji = "🟡"
    elif update_ratio <= 1e-3:
        status = "optimal"  # 🟢 оптимальный диапазон
        numeric_status = 3.0
        emoji = "🟢"
    else:
        status = "large"  # 🟠 слишком большие обновления
        numeric_status = 4.0
        emoji = "🟠"
        
    return {
        "update_ratio": float(update_ratio),
        "update_ratio_status": status,
        "update_ratio_numeric": numeric_status,
        "update_ratio_emoji": emoji,
        "grad_norm": float(grad_norm),
        "weight_norm": float(weight_norm)
    }

def calculate_separate_update_ratios(projector, gemma_model):
    """
    📊 Вычисляет update ratio отдельно для проектора и LoRA адаптеров
    
    Returns:
        dict: Отдельные метрики для каждого компонента
    """
    metrics = {}
    
    # 🎯 Update ratio для проектора
    proj_grad_norm = 0.0
    proj_weight_norm = 0.0
    
    for param in projector.parameters():
        if param.grad is not None:
            proj_grad_norm += param.grad.data.norm(2).item() ** 2
        proj_weight_norm += param.data.norm(2).item() ** 2
    
    proj_grad_norm = proj_grad_norm ** 0.5
    proj_weight_norm = proj_weight_norm ** 0.5
    
    if proj_grad_norm > 0:
        proj_metrics = calculate_update_ratio_metrics(proj_grad_norm, proj_weight_norm)
        metrics['projector'] = proj_metrics
    
    # 🔧 Update ratio для LoRA адаптеров
    lora_grad_norm = 0.0
    lora_weight_norm = 0.0
    
    for param in gemma_model.parameters():
        if param.requires_grad and param.grad is not None:
            lora_grad_norm += param.grad.data.norm(2).item() ** 2
            lora_weight_norm += param.data.norm(2).item() ** 2
    
    lora_grad_norm = lora_grad_norm ** 0.5
    lora_weight_norm = lora_weight_norm ** 0.5
    
    if lora_grad_norm > 0:
        lora_metrics = calculate_update_ratio_metrics(lora_grad_norm, lora_weight_norm)
        metrics['lora'] = lora_metrics
    
    return metrics


In [ ]:
# 🧪 Функция тестирования анализатора
def test_embedding_analyzer(analyzer, test_batch, compression_rate_k, prefix_embeds):
    """
    🧪 Тестирует анализатор embedding'ов на одном батче
    
    Args:
        analyzer: экземпляр AudioTextEmbeddingAnalyzer
        test_batch: тестовый батч данных
        compression_rate_k: коэффициент сжатия аудио
        prefix_embeds: префикс embedding'ы
    """
    if analyzer is None:
        print("❌ Анализатор не инициализирован")
        return
        
    print("🧪 Запуск тестирования анализатора embedding'ов...")
    
    try:
        # Запускаем комплексный анализ
        results = analyzer.run_comprehensive_analysis(
            batch=test_batch,
            compression_rate_k=compression_rate_k,
            prefix_embeds=prefix_embeds,
            sample_name="test"
        )
        
        print("\n✅ Тест завершен успешно!")
        print(f"📊 Получены результаты: {list(results.keys())}")
        
        # Показываем строку ближайших токенов
        if 'tokens_string' in results:
            tokens_data = results['tokens_string']
            print(f"\n🎯 ТЕСТ: Аудио → MLP → Токены")
            print(f"   Строка: '{tokens_data['nearest_tokens_readable']}'")
            print(f"   Similarity: {tokens_data['statistics']['avg_similarity']:.3f}")
        
        return results
        
    except Exception as e:
        print(f"❌ Ошибка тестирования: {e}")
        return None

print("✅ Функция test_embedding_analyzer готова!")
print("🔧 Использование: test_embedding_analyzer(embedding_analyzer, batch, compression_rate_k, prefix_embeds)")


In [ ]:
# 🔬 Импорт анализатора из отдельного файла
import sys
sys.path.append('.')

try:
    from audio_analysis_standalone import AudioTextEmbeddingAnalyzer
    print("🔬 Класс AudioTextEmbeddingAnalyzer импортирован из audio_analysis_standalone.py")
    print("📊 Функции: поиск ближайших токенов, t-SNE, матрица косинусной близости, анализ выравнивания")
    print("🔧 ИСПРАВЛЕНО: Все операции переведены в float32 для совместимости с BFloat16")
except ImportError as e:
    print(f"⚠️ Не удалось импортировать анализатор: {e}")
    print("🔧 Создайте файл audio_analysis_standalone.py или отключите анализ")
    AudioTextEmbeddingAnalyzer = None


In [ ]:
# 🧪 Функция тестирования анализатора (обновленная)
def test_embedding_analyzer(analyzer, test_batch, compression_rate_k, prefix_embeds):
    """
    🧪 Тестирует анализатор embedding'ов на одном батче
    🔧 ИСПРАВЛЕНО: Теперь использует внешний анализатор с поддержкой float32
    """
    if analyzer is None:
        print("❌ Анализатор не инициализирован")
        return
        
    print("🧪 Запуск тестирования анализатора embedding'ов...")
    
    try:
        # Запускаем комплексный анализ
        results = analyzer.run_comprehensive_analysis(
            batch=test_batch,
            compression_rate_k=compression_rate_k,
            prefix_embeds=prefix_embeds,
            current_step=0
        )
        
        print("\n✅ Тест завершен успешно!")
        print(f"📊 Получены результаты: {list(results.keys())}")
        
        # Показываем строку ближайших токенов
        if 'tokens_string' in results:
            tokens_data = results['tokens_string']
            print(f"\n🎯 ТЕСТ: Аудио → MLP → Токены")
            print(f"   Строка: '{tokens_data['nearest_tokens_readable']}'")
            print(f"   Similarity: {tokens_data['statistics']['avg_similarity']:.3f}")
        
        return results
        
    except Exception as e:
        print(f"❌ Ошибка тестирования: {e}")
        return None

print("✅ Функция test_embedding_analyzer готова!")


In [ ]:
# 🔬 Анализатор перенесен в отдельный файл audio_analysis_standalone.py
# Этот код больше не нужен в основном файле обучения
print("✅ Анализ embedding'ов вынесен в отдельный модуль для исправления BFloat16 ошибки")
        self.projector = projector
        self.wav2vec2 = wav2vec2
        self.tokenizer = tokenizer
        self.device = device
        self.save_to_wandb = save_to_wandb  # Сохранять графики в wandb вместо локальной папки
        
        # Создаем vocabulary embedding matrix для поиска ближайших токенов
        self._create_vocab_embeddings()
        
    def _create_vocab_embeddings(self):
        """Создает матрицу embedding'ов для всего словаря"""
        print("🔤 Создание матрицы embedding'ов словаря...")
        vocab_size = self.tokenizer.vocab_size
        
        # Берем первые 10000 токенов для ускорения (можно увеличить)
        max_tokens = min(10000, vocab_size)
        token_ids = torch.arange(max_tokens, device=self.device)
        
        with torch.no_grad():
            self.vocab_embeddings = self.model.get_input_embeddings()(token_ids)  # [vocab_size, hidden_dim]
            
        # Создаем mapping token_id -> token_text для интерпретации
        self.token_id_to_text = {}
        for i in range(max_tokens):
            try:
                token_text = self.tokenizer.decode([i], skip_special_tokens=False)
                self.token_id_to_text[i] = token_text
            except:
                self.token_id_to_text[i] = f"<UNK_{i}>"
                
        print(f"✅ Создана матрица embedding'ов для {max_tokens} токенов")
    
    def find_nearest_tokens(self, projected_audio_embeds: torch.Tensor, top_k: int = 10) -> List[Dict]:
        """
        Находит ближайшие токены для каждого audio embedding'а по косинусной близости
        
        Args:
            projected_audio_embeds: [seq_len, hidden_dim] - проектированные аудио embedding'ы
            top_k: количество ближайших токенов для поиска
            
        Returns:
            List[Dict]: список с информацией о ближайших токенах для каждого временного шага
        """
        with torch.no_grad():
            # Нормализуем embedding'ы для косинусной близости
            audio_norm = F.normalize(projected_audio_embeds, p=2, dim=-1)  # [seq_len, hidden_dim]
            vocab_norm = F.normalize(self.vocab_embeddings, p=2, dim=-1)   # [vocab_size, hidden_dim]
            
            # Вычисляем косинусную близость: [seq_len, vocab_size]
            similarity_matrix = torch.mm(audio_norm, vocab_norm.t())
            
            # Находим top_k для каждого временного шага
            top_similarities, top_indices = torch.topk(similarity_matrix, top_k, dim=-1)
            
            results = []
            for t in range(projected_audio_embeds.size(0)):
                step_result = {
                    'timestep': t,
                    'nearest_tokens': [],
                    'similarities': top_similarities[t].cpu().numpy().tolist(),
                    'token_ids': top_indices[t].cpu().numpy().tolist()
                }
                
                for i in range(top_k):
                    token_id = top_indices[t, i].item()
                    similarity = top_similarities[t, i].item()
                    token_text = self.token_id_to_text.get(token_id, f"<UNK_{token_id}>")
                    
                    step_result['nearest_tokens'].append({
                        'token_id': token_id,
                        'token_text': token_text,
                        'similarity': similarity
                    })
                
                results.append(step_result)
                
        return results
    
    def create_tsne_visualization(self, 
                                  projected_audio_list: List[torch.Tensor], 
                                  target_embeds_list: List[torch.Tensor],
                                  labels: List[str] = None,
                                  save_name: str = "tsne_embeddings.png"):
        """
        Создает t-SNE визуализацию audio и text embedding'ов
        
        Args:
            projected_audio_list: список проектированных аудио embedding'ов
            target_embeds_list: список целевых текстовых embedding'ов  
            labels: подписи для каждого примера
            save_name: имя файла для сохранения
        """
        print("📊 Создание t-SNE визуализации...")
        
        # Объединяем все embedding'ы
        all_embeddings = []
        all_types = []
        all_labels = []
        
        for i, (audio_emb, text_emb) in enumerate(zip(projected_audio_list, target_embeds_list)):
            # Усредняем по временным шагам для упрощения
            audio_mean = audio_emb.mean(dim=0).cpu().numpy()
            text_mean = text_emb.mean(dim=0).cpu().numpy()
            
            all_embeddings.extend([audio_mean, text_mean])
            all_types.extend(['Audio', 'Text'])
            
            label = labels[i] if labels else f"Sample_{i}"
            all_labels.extend([f"Audio_{label}", f"Text_{label}"])
        
        # Применяем t-SNE
        embeddings_array = np.array(all_embeddings)
        
        # Сначала PCA для уменьшения размерности (ускоряет t-SNE)
        if embeddings_array.shape[1] > 50:
            pca = PCA(n_components=50)
            embeddings_array = pca.fit_transform(embeddings_array)
            
        tsne = TSNE(n_components=2, random_state=42, perplexity=min(30, len(all_embeddings)//4))
        embeddings_2d = tsne.fit_transform(embeddings_array)
        
        # Создаем DataFrame для seaborn
        df = pd.DataFrame({
            'x': embeddings_2d[:, 0],
            'y': embeddings_2d[:, 1], 
            'type': all_types,
            'label': all_labels
        })
        
        # Создаем визуализацию
        plt.figure(figsize=(12, 8))
        sns.scatterplot(data=df, x='x', y='y', hue='type', style='type', s=100, alpha=0.7)
        
        plt.title("t-SNE: Audio vs Text Embeddings", fontsize=16)
        plt.xlabel("t-SNE Component 1", fontsize=12)
        plt.ylabel("t-SNE Component 2", fontsize=12)
        plt.legend(title="Embedding Type", fontsize=12)
        plt.grid(True, alpha=0.3)
        
        # Сохраняем в wandb или показываем локально
        plt.tight_layout()
        
        if self.save_to_wandb:
            # Логируем в wandb
            import wandb
            wandb.log({"analysis/tsne_audio_text_embeddings": wandb.Image(plt)})
            print("✅ t-SNE визуализация отправлена в Weights & Biases")
        else:
            plt.show()
            print("✅ t-SNE визуализация отображена локально")
        
        plt.close()  # Освобождаем память
        return df
    
    def create_similarity_heatmap(self, 
                                  projected_audio: torch.Tensor, 
                                  target_embeds: torch.Tensor,
                                  save_name: str = "similarity_heatmap.png",
                                  title: str = "Audio-Text Cosine Similarity"):
        """
        Создает матрицу попарной косинусной близости (как на картинке из статьи)
        
        Args:
            projected_audio: [audio_seq_len, hidden_dim] - проектированные аудио embedding'ы
            target_embeds: [text_seq_len, hidden_dim] - текстовые embedding'ы
            save_name: имя файла для сохранения
            title: заголовок графика
        """
        print("🔥 Создание матрицы косинусной близости...")
        
        with torch.no_grad():
            # Нормализуем для косинусной близости
            audio_norm = F.normalize(projected_audio, p=2, dim=-1)  # [audio_len, hidden_dim]
            text_norm = F.normalize(target_embeds, p=2, dim=-1)     # [text_len, hidden_dim]
            
            # Вычисляем попарную косинусную близость
            similarity_matrix = torch.mm(audio_norm, text_norm.t())  # [audio_len, text_len]
            similarity_np = similarity_matrix.cpu().numpy()
        
        # Создаем тепловую карту
        plt.figure(figsize=(12, 8))
        
        # Используем цветовую схему как на картинке (синий-фиолетовый-желтый)
        sns.heatmap(similarity_np, 
                   cmap='viridis',  # или 'plasma' для более точного соответствия картинке
                   cbar=True,
                   xticklabels=False,
                   yticklabels=False,
                   cbar_kws={'label': 'Cosine Similarity'})
        
        plt.title(title, fontsize=16)
        plt.xlabel('Text Tokens', fontsize=12)
        plt.ylabel('Audio Tokens', fontsize=12)
        
        # Добавляем colorbar statistics
        plt.figtext(0.02, 0.02, f'Min: {similarity_np.min():.3f}, Max: {similarity_np.max():.3f}, Mean: {similarity_np.mean():.3f}', 
                   fontsize=10, ha='left')
        
        # Сохраняем в wandb или показываем локально
        plt.tight_layout()
        
        if self.save_to_wandb:
            # Логируем в wandb
            import wandb
            wandb.log({
                "analysis/cosine_similarity_heatmap": wandb.Image(plt),
                "analysis/similarity_min": float(similarity_np.min()),
                "analysis/similarity_max": float(similarity_np.max()),
                "analysis/similarity_mean": float(similarity_np.mean()),
                "analysis/similarity_std": float(similarity_np.std())
            })
            print("✅ Матрица косинусной близости отправлена в Weights & Biases")
        else:
            plt.show()
            print("✅ Матрица косинусной близости отображена локально")
        
        plt.close()  # Освобождаем память
        
        # Возвращаем статистики
        return {
            'similarity_matrix': similarity_np,
            'min_similarity': float(similarity_np.min()),
            'max_similarity': float(similarity_np.max()),
            'mean_similarity': float(similarity_np.mean()),
            'std_similarity': float(similarity_np.std())
        }
    
    def analyze_alignment_quality(self, 
                                  projected_audio: torch.Tensor, 
                                  target_embeds: torch.Tensor,
                                  text_tokens: List[str] = None) -> Dict:
        """
        Анализирует качество выравнивания между аудио и текстом
        
        Args:
            projected_audio: [audio_seq_len, hidden_dim]
            target_embeds: [text_seq_len, hidden_dim] 
            text_tokens: список текстовых токенов для анализа
            
        Returns:
            Dict с метриками качества выравнивания
        """
        with torch.no_grad():
            # Косинусная близость
            audio_norm = F.normalize(projected_audio, p=2, dim=-1)
            text_norm = F.normalize(target_embeds, p=2, dim=-1)
            similarity_matrix = torch.mm(audio_norm, text_norm.t())
            
            # Diagonal alignment score (идеальное выравнивание по диагонали)
            min_len = min(similarity_matrix.size(0), similarity_matrix.size(1))
            if min_len > 1:
                diagonal_similarities = torch.diag(similarity_matrix[:min_len, :min_len])
                diagonal_score = diagonal_similarities.mean().item()
            else:
                diagonal_score = 0.0
            
            # Maximum alignment score (лучшая возможная match для каждого audio token)
            max_similarities_per_audio = similarity_matrix.max(dim=1)[0]
            max_alignment_score = max_similarities_per_audio.mean().item()
            
            # Attention-like weights (softmax по text dimension)
            attention_weights = F.softmax(similarity_matrix * 10, dim=1)  # temperature=0.1
            attention_entropy = -(attention_weights * torch.log(attention_weights + 1e-8)).sum(dim=1).mean().item()
            
        metrics = {
            'diagonal_alignment_score': diagonal_score,
            'max_alignment_score': max_alignment_score,
            'attention_entropy': attention_entropy,
            'similarity_stats': {
                'min': float(similarity_matrix.min()),
                'max': float(similarity_matrix.max()),
                'mean': float(similarity_matrix.mean()),
                'std': float(similarity_matrix.std())
            }
        }
        
        return metrics
    
    def run_comprehensive_analysis(self, 
                                   batch: Dict,
                                   compression_rate_k: int,
                                   prefix_embeds: torch.Tensor,
                                   sample_name: str = "analysis") -> Dict:
        """
        Запускает комплексный анализ для одного батча
        
        Args:
            batch: батч данных (audio + text)
            compression_rate_k: коэффициент сжатия аудио
            prefix_embeds: префикс embedding'ы
            sample_name: имя для сохранения файлов
            
        Returns:
            Dict с результатами всех анализов
        """
        print(f"🔬 Запуск комплексного анализа для {sample_name}...")
        
        # Переводим модели в eval режим
        self.projector.eval()
        self.wav2vec2.eval()
        self.model.eval()
        
        with torch.no_grad():
            # Получаем embedding'ы
            input_values = batch["input_values"].to(self.device, dtype=torch.bfloat16)
            input_ids = batch["input_ids"].to(self.device)
            
            # Аудио pipeline
            audio_embeds = self.wav2vec2(input_values).last_hidden_state
            compressed_audio = self._compress_audio_features(audio_embeds, compression_rate_k)
            projected_audio = self.projector(compressed_audio).to(torch.bfloat16)
            
            # Текстовые embedding'ы
            embedding_input_ids = input_ids.clone()
            embedding_input_ids[embedding_input_ids == -100] = self.tokenizer.pad_token_id
            target_embeds = self.model.get_input_embeddings()(embedding_input_ids)
            
            # Получаем текстовые токены для анализа
            text_tokens = []
            for ids in input_ids:
                valid_ids = ids[ids != -100]
                tokens = [self.tokenizer.decode([id.item()]) for id in valid_ids]
                text_tokens.extend(tokens)
        
        results = {}
        
        # 1. Поиск ближайших токенов
        print("🔍 Анализ ближайших токенов...")
        # Берем первый пример из батча
        first_projected = projected_audio[0]  # [seq_len, hidden_dim]
        nearest_tokens = self.find_nearest_tokens(first_projected, top_k=5)
        results['nearest_tokens'] = nearest_tokens[:10]  # Первые 10 временных шагов
        
        # 🎯 НОВОЕ: Создание строки ближайших токенов (то, что хотел пользователь!)
        print("🎯 Создание строки перевода: аудио → MLP → ближайшие токены...")
        tokens_string_result = self.create_nearest_tokens_string(first_projected, max_length=30)
        results['tokens_string'] = tokens_string_result
        
        # Выводим результат для пользователя
        print(f"\n📝 АУДИО → MLP → ТОКЕНЫ:")
        print(f"   Сырая строка: '{tokens_string_result['nearest_tokens_raw']}'")
        print(f"   Читаемая версия: '{tokens_string_result['nearest_tokens_readable']}'")
        print(f"   Средняя similarity: {tokens_string_result['statistics']['avg_similarity']:.3f}")
        print(f"   Длина последовательности: {tokens_string_result['statistics']['sequence_length']} токенов")
        
        # Показываем первые несколько токенов с similarity scores
        print(f"\n🔍 Детали первых токенов:")
        for detail in tokens_string_result['token_details'][:5]:
            print(f"   Шаг {detail['timestep']}: '{detail['token']}' (similarity: {detail['similarity']:.3f})")
        if len(tokens_string_result['token_details']) > 5:
            print(f"   ... и еще {len(tokens_string_result['token_details']) - 5} токенов")
        
        # 2. Создаем матрицу косинусной близости
        print("🔥 Создание матрицы косинусной близости...")
        first_target = target_embeds[0]  # [seq_len, hidden_dim]
        similarity_stats = self.create_similarity_heatmap(
            first_projected, 
            first_target,
            save_name=f"similarity_heatmap_{sample_name}.png",
            title=f"Audio-Text Similarity: {sample_name}"
        )
        results['similarity_stats'] = similarity_stats
        
        # 3. Анализ качества выравнивания
        print("📊 Анализ качества выравнивания...")
        alignment_metrics = self.analyze_alignment_quality(
            first_projected,
            first_target,
            text_tokens
        )
        results['alignment_metrics'] = alignment_metrics
        
        # 4. t-SNE визуализация (если больше одного примера в батче)
        if projected_audio.size(0) > 1:
            print("📈 Создание t-SNE визуализации...")
            projected_list = [projected_audio[i] for i in range(min(5, projected_audio.size(0)))]
            target_list = [target_embeds[i] for i in range(min(5, target_embeds.size(0)))]
            labels = [f"Sample_{i}" for i in range(len(projected_list))]
            
            tsne_df = self.create_tsne_visualization(
                projected_list,
                target_list, 
                labels=labels,
                save_name=f"tsne_{sample_name}.png"
            )
            results['tsne_data'] = tsne_df.to_dict()
        
        print(f"✅ Комплексный анализ завершен для {sample_name}")
        return results
    
    def create_nearest_tokens_string(self, projected_audio_embeds: torch.Tensor, 
                                   max_length: int = 50) -> Dict:
        """
        🎯 Создает строку из ближайших токенов для каждого audio embedding'а
        Это то, что хотел пользователь - визуализация "перевода" аудио через MLP в токены
        
        Args:
            projected_audio_embeds: [seq_len, hidden_dim] - проектированные аудио embedding'ы
            max_length: максимальная длина результирующей строки
            
        Returns:
            Dict с информацией о "переводе" аудио в токены
        """
        print("🎯 Создание строки ближайших токенов (аудио → токены через MLP)...")
        
        with torch.no_grad():
            # Нормализуем embedding'ы для косинусной близости
            audio_norm = F.normalize(projected_audio_embeds, p=2, dim=-1)  # [seq_len, hidden_dim]
            vocab_norm = F.normalize(self.vocab_embeddings, p=2, dim=-1)   # [vocab_size, hidden_dim]
            
            # Вычисляем косинусную близость: [seq_len, vocab_size]
            similarity_matrix = torch.mm(audio_norm, vocab_norm.t())
            
            # Находим самый близкий токен для каждого временного шага
            top_similarities, top_indices = torch.topk(similarity_matrix, 1, dim=-1)
            
            # Создаем строку из ближайших токенов
            nearest_tokens_list = []
            similarities_list = []
            
            seq_len = min(projected_audio_embeds.size(0), max_length)
            
            for t in range(seq_len):
                token_id = top_indices[t, 0].item()
                similarity = top_similarities[t, 0].item()
                token_text = self.token_id_to_text.get(token_id, f"<UNK_{token_id}>")
                
                nearest_tokens_list.append(token_text)
                similarities_list.append(similarity)
            
            # Создаем результирующую строку
            nearest_tokens_string = "".join(nearest_tokens_list)
            
            # Также создаем версию с пробелами для лучшей читаемости
            readable_string = " ".join([token.strip() for token in nearest_tokens_list if token.strip()])
            
            # Статистики
            avg_similarity = sum(similarities_list) / len(similarities_list)
            min_similarity = min(similarities_list)
            max_similarity = max(similarities_list)
            
            result = {
                'nearest_tokens_raw': nearest_tokens_string,
                'nearest_tokens_readable': readable_string,
                'individual_tokens': nearest_tokens_list,
                'similarities': similarities_list,
                'statistics': {
                    'avg_similarity': avg_similarity,
                    'min_similarity': min_similarity,
                    'max_similarity': max_similarity,
                    'sequence_length': seq_len
                },
                'token_details': [
                    {
                        'timestep': t,
                        'token': nearest_tokens_list[t],
                        'similarity': similarities_list[t]
                    }
                    for t in range(seq_len)
                ]
            }
            
        return result
    
    def _compress_audio_features(self, audio_features, compression_rate_k):
        """Вспомогательная функция сжатия аудио (копия из основного кода)"""
        batch_size, seq_len, hidden_dim = audio_features.shape
        
        new_seq_len = (seq_len // compression_rate_k) * compression_rate_k
        audio_features = audio_features[:, :new_seq_len, :]
        
        reshaped = audio_features.view(batch_size, new_seq_len // compression_rate_k, compression_rate_k, hidden_dim)
        compressed = reshaped.view(batch_size, new_seq_len // compression_rate_k, compression_rate_k * hidden_dim)
        
        return compressed

print("🔬 Класс AudioTextEmbeddingAnalyzer готов к использованию!")
print("📊 Функции: поиск ближайших токенов, t-SNE, матрица косинусной близости, анализ выравнивания")


In [ ]:
import sys
from tqdm import tqdm
import subprocess

def install_deps(package_file):
    try:
        with open(package_file, 'r') as f:
            packages = [line.strip() for line in f if line.strip() and not line.startswith('#')]
    except FileNotFoundError:
        return
    
    for package in tqdm(packages, desc="📥 Обновление пакетов"):
        try:
            subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", package, "-q"], check=True, capture_output=True)
        except subprocess.CalledProcessError:
            pass
    
    print("✅ Dependencies updated")

install_deps("requirements.txt")

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
import torchaudio
import json
import os
import gc
import numpy as np
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.amp import autocast, GradScaler
from transformers import AutoConfig, AutoTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Model
from transformers.utils.quantization_config import BitsAndBytesConfig
from transformers.models.gemma3.modeling_gemma3 import Gemma3ForCausalLM
from transformers.models.gemma3.configuration_gemma3 import Gemma3TextConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import jiwer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from huggingface_hub import notebook_login, login
from sklearn.model_selection import train_test_split
import pandas as pd
from datetime import datetime
from torch.optim.lr_scheduler import OneCycleLR, CosineAnnealingWarmRestarts
from IPython.display import Audio, display
import zipfile
import io
import wandb
import glob
import random
import bitsandbytes as bnb

def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

In [ ]:
def collate_fn(batch):
    input_values = [item['input_values'] for item in batch]
    input_ids = [item['input_ids'] for item in batch]
    attention_mask = [item['attention_mask'] for item in batch]
    input_values = pad_sequence(input_values, batch_first=True)
    input_ids = pad_sequence(input_ids, batch_first=True, padding_value=-100)
    attention_mask = pad_sequence(attention_mask, batch_first=True, padding_value=0)
    return {
        'input_values': input_values,
        'input_ids': input_ids,
        'attention_mask': attention_mask
    }

def process_batch(batch, model, projector, wav2vec2, tokenizer, prefix_embeds, device, compression_rate_k, ctc_head=None, ctc_loss_fn=None, lambda_ctc=0.1):
    input_values = batch["input_values"].to(device, dtype=torch.bfloat16)
    input_ids = batch["input_ids"].to(device)

    with autocast('cuda' if torch.cuda.is_available() else 'cpu'):
        audio_embeds = wav2vec2(input_values).last_hidden_state
        
        # 🔤 CTC Loss: вычисляем до compression для лучшего разрешения
        ctc_loss = 0.0
        if ctc_head is not None and ctc_loss_fn is not None:
            # Извлекаем оригинальные тексты из токенов для CTC целей
            text_batch = []
            for ids in input_ids:
                # Убираем -100 (padding токены)
                valid_ids = ids[ids != -100]
                text = tokenizer.decode(valid_ids, skip_special_tokens=True)
                text_batch.append(text)
            
            # CTC forward через аудио эмбеддинги
            ctc_log_probs = ctc_head(audio_embeds)  # [batch, time, vocab]
            
            # Конвертируем тексты в CTC цели
            ctc_targets, target_lengths = text_to_ctc_targets(text_batch)
            ctc_targets = ctc_targets.to(device)
            target_lengths = target_lengths.to(device)
            
            # Длины входных последовательностей для CTC
            input_lengths = torch.full((audio_embeds.size(0),), audio_embeds.size(1), dtype=torch.long, device=device)
            
            # CTC Loss
            try:
                ctc_loss = ctc_loss_fn(
                    ctc_log_probs.transpose(0, 1),  # [time, batch, vocab] для CTC
                    ctc_targets,
                    input_lengths,
                    target_lengths
                )
                ctc_loss = ctc_loss * lambda_ctc  # Взвешиваем CTC loss
            except Exception as e:
                print(f"⚠️ CTC Loss ошибка: {e}")
                ctc_loss = 0.0
        
        compressed_audio = compress_audio_features(audio_embeds, compression_rate_k)
        del audio_embeds  # Освобождаем память
        
        projected_audio = projector(compressed_audio)
        del compressed_audio  # Освобождаем память
        # Ensure projected audio is in bfloat16 for consistency
        projected_audio = projected_audio.to(dtype=torch.bfloat16)
        
        batch_prefix_embeds = prefix_embeds.expand(projected_audio.size(0), -1, -1)
        
        prompt_embeds = torch.cat([batch_prefix_embeds, projected_audio], dim=1)
        
        embedding_input_ids = input_ids.clone()
        embedding_input_ids[embedding_input_ids == -100] = tokenizer.pad_token_id
        target_embeds = model.get_input_embeddings()(embedding_input_ids)
        del embedding_input_ids  # Освобождаем память

        # 🔧 ДИАГНОСТИКА: Вычисляем L2-нормы префикса и токенов
        prefix_l2_norm = torch.norm(batch_prefix_embeds, p=2, dim=-1).mean().item()
        projected_l2_norm = torch.norm(projected_audio, p=2, dim=-1).mean().item() 
        target_l2_norm = torch.norm(target_embeds, p=2, dim=-1).mean().item()
        
        norm_ratio = projected_l2_norm / max(target_l2_norm, 1e-8)

        inputs_embeds = torch.cat([prompt_embeds, target_embeds], dim=1)
        del target_embeds  # Освобождаем память
        
        prompt_len = prompt_embeds.shape[1]
        prompt_labels = torch.full((projected_audio.size(0), prompt_len), -100, device=device, dtype=torch.long)
        del projected_audio  # Освобождаем память
        
        labels = torch.cat([prompt_labels, input_ids], dim=1)
        del prompt_labels  # Освобождаем память

        outputs = model(inputs_embeds=inputs_embeds, labels=labels)
        del inputs_embeds, labels  # Освобождаем память
        
        # 🔤 Добавляем CTC loss к основному loss
        if ctc_loss > 0:
            outputs.loss = outputs.loss + ctc_loss
        
    # Возвращаем диагностические данные
    norm_diagnostics = {
        'prefix_l2_norm': prefix_l2_norm,
        'projected_l2_norm': projected_l2_norm, 
        'target_l2_norm': target_l2_norm,
        'norm_ratio': norm_ratio,
        'ctc_loss': float(ctc_loss) if isinstance(ctc_loss, torch.Tensor) else ctc_loss
    }
    
    return outputs, prompt_embeds, norm_diagnostics

In [ ]:
def force_gpu_cleanup():
    """Принудительная очистка GPU памяти"""
    import gc
    import torch
    
    # 1. Удаляем все известные переменные
    variables_to_delete = [
        'gemma_model', 'wav2vec2', 'projector', 'train_loader', 'val_loader',
        'optimizer', 'scheduler', 'scaler', 'train_dataset', 'val_dataset',
        'prefix_embeds', 'tokenizer', 'feature_extractor', 'logger',
        'quantization_config', 'lora_config'
    ]
    
    deleted_count = 0
    for var_name in variables_to_delete:
        if var_name in globals():
            try:
                # Для моделей PyTorch - перемещаем на CPU перед удалением
                obj = globals()[var_name]
                if hasattr(obj, 'cpu'):
                    obj.cpu()
                if hasattr(obj, 'to'):
                    obj.to('cpu')
                del globals()[var_name]
                deleted_count += 1
            except:
                pass
    
    # 2. Удаляем все тензоры из кэша
    torch._C._cuda_clearCublasWorkspaces()
    
    # 3. Очищаем кэш автоградов
    if hasattr(torch.autograd, 'set_grad_enabled'):
        torch.autograd.set_grad_enabled(False)
        torch.autograd.set_grad_enabled(True)
    
    # 4. Принудительный сбор мусора (несколько раз)
    for _ in range(5):
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
    
    # 5. Очистка GPU памяти
    if torch.cuda.is_available():
        # Синхронизируем все операции
        torch.cuda.synchronize()
        
        # Очищаем все кэши
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()
        
        # Сбрасываем статистики
        torch.cuda.reset_peak_memory_stats()
        torch.cuda.reset_accumulated_memory_stats()
        
        # Финальный сбор мусора
        gc.collect()
        
        # Получаем статистику памяти
        allocated = torch.cuda.memory_allocated() / 1024**3
        reserved = torch.cuda.memory_reserved() / 1024**3
        
        print(f"🧹 Принудительная очистка GPU памяти:")
        print(f"   📊 Удалено переменных: {deleted_count}")
        print(f"   💾 Выделено сейчас: {allocated:.2f} GB")
        print(f"   🔒 Зарезервировано: {reserved:.2f} GB")
        
        return allocated, reserved
    else:
        print(f"🧹 CPU очистка завершена (удалено переменных: {deleted_count})")
        return 0, 0

# Запускаем принудительную очистку
force_gpu_cleanup()

In [ ]:
def load_checkpoint(path, projector, gemma_model, optimizer, scheduler, device, batch_size):
    global best_val_loss
    checkpoint = torch.load(path, map_location=device)
    
    try:
        projector.load_state_dict(checkpoint['projector_state_dict'])
    except RuntimeError as e:
        print(f"⚠️ Несовместимость проектора: {e}")
        print("🔄 Инициализация проектора новыми весами, т.к. конфигурация (напр. compression_rate_k) изменилась.")
        wandb.log({"checkpoint/projector_reinitialized": True})
    
    if 'lora_state_dict' in checkpoint:
        gemma_model.load_state_dict(checkpoint['lora_state_dict'], strict=False)
    
    print("🔄 Пропускаем загрузку состояния оптимизатора для экономии памяти. Он будет инициализирован заново.")
    wandb.log({"checkpoint/optimizer_reset_manual": True})
    
    try:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    except Exception as e:
        print(f"⚠️ Scheduler error: {e}")
    
    start_epoch = checkpoint['epoch']
    saved_step = checkpoint['step']
    best_val_loss = checkpoint.get('best_val_loss', float('inf'))
    
    # ОБРАТНАЯ СОВМЕСТИМОСТЬ: config может не быть в старых чекпоинтах
    config = checkpoint.get('config', {})
    prev_batch_size = config.get('batch_size', batch_size)
    
    # АВТОМАТИЧЕСКИЙ РАСЧЕТ batch_idx для старых чекпоинтов
    if 'batch_idx' in checkpoint:
        # Новый чекпоинт - берем сохраненный batch_idx
        batch_idx = checkpoint['batch_idx']
        checkpoint_version = "new"
    else:
        # Старый чекпоинт - вычисляем batch_idx автоматически
        checkpoint_version = "legacy"
        
        # Метод 1: Пытаемся извлечь из имени файла (latest_checkpoint_bs4_epoch_4_step_5500.pt)
        import re
        filename = os.path.basename(path)
        match = re.search(r'epoch_(\d+)_step_(\d+)', filename)
        
        if match:
            file_epoch = int(match.group(1))
            file_step = int(match.group(2))
            
            # ПРАВИЛЬНЫЙ РАСЧЕТ: batch_idx должен быть позицией в ТЕКУЩЕЙ эпохе
            # У нас есть данные о том, сколько батчей в эпохе
            batches_per_epoch = len(train_data) // batch_size if 'train_data' in globals() else 2000
            
            # Если мы в той же эпохе, то batch_idx = сколько батчей уже прошло в этой эпохе
            # start_epoch это эпоха, с которой мы возобновляем (уже установлена равной file_epoch - 1)
            if start_epoch == file_epoch - 1:  # Мы в той же эпохе, что и чекпоинт
                # batch_idx = остаток от деления global_step на batches_per_epoch
                batch_idx = file_step % batches_per_epoch
            else:
                # Если эпохи не совпадают, начинаем сначала
                batch_idx = 0
            
            print(f"📦 Legacy чекпоинт: извлечено из имени файла epoch={file_epoch}, step={file_step}")
            print(f"📊 Батчей в эпохе: {batches_per_epoch}")
            print(f"📊 Позиция в эпохе {file_epoch}: batch_idx = {file_step} % {batches_per_epoch} = {batch_idx}")
        else:
            # Метод 2: Используем start_epoch и saved_step для правильного расчета
            steps_per_epoch_estimate = len(train_data) // batch_size if 'train_data' in globals() else 2000
            
            # Сколько полных эпох прошло
            completed_epochs = start_epoch
            steps_in_completed_epochs = completed_epochs * steps_per_epoch_estimate
            
            # batch_idx = позиция в текущей эпохе
            batch_idx = saved_step - steps_in_completed_epochs
            
            # Проверяем, что batch_idx в разумных пределах
            if batch_idx < 0:
                batch_idx = 0
            elif batch_idx >= steps_per_epoch_estimate:
                batch_idx = steps_per_epoch_estimate - 1
            
            print(f"📦 Legacy чекпоинт: не удалось извлечь из имени, используем saved_step={saved_step}")
            print(f"📊 Расчет: {completed_epochs} полных эпох × {steps_per_epoch_estimate} = {steps_in_completed_epochs} шагов")
            print(f"📊 Позиция в эпохе {start_epoch + 1}: batch_idx = {saved_step} - {steps_in_completed_epochs} = {batch_idx}")
        
        wandb.log({
            "checkpoint/legacy_batch_idx_calculated": True,
            "checkpoint/calculated_batch_idx": batch_idx,
            "checkpoint/filename": filename
        })
    
    if prev_batch_size != batch_size:
        total_samples_seen = saved_step * prev_batch_size
        adjusted_step = total_samples_seen // batch_size
        
        # Пересчитываем batch_idx при изменении batch_size
        if checkpoint_version == "legacy":
            steps_per_epoch_estimate = len(train_data) // batch_size if 'train_data' in globals() else 2000
            
            # Правильно вычисляем позицию в текущей эпохе для adjusted_step
            completed_epochs = start_epoch
            steps_in_completed_epochs = completed_epochs * steps_per_epoch_estimate
            batch_idx = adjusted_step - steps_in_completed_epochs
            
            # Проверяем границы
            if batch_idx < 0:
                batch_idx = 0
            elif batch_idx >= steps_per_epoch_estimate:
                batch_idx = steps_per_epoch_estimate - 1
        
        wandb.log({
            "checkpoint/batch_size_mismatch": True,
            "checkpoint/prev_batch_size": prev_batch_size,
            "checkpoint/new_batch_size": batch_size,
            "checkpoint/samples_seen": total_samples_seen,
            "checkpoint/adjusted_step": adjusted_step,
            "checkpoint/adjusted_batch_idx": batch_idx
        })
        
        global_step = adjusted_step
    else:
        global_step = saved_step
    
    wandb.log({
        "checkpoint/loaded": True,
        "checkpoint/version": checkpoint_version,
        "checkpoint/start_epoch": start_epoch,
        "checkpoint/global_step": global_step,
        "checkpoint/best_val_loss": best_val_loss,
        "checkpoint/batch_idx": batch_idx
    })
    
    if checkpoint_version == "legacy":
        print(f"📦 Legacy чекпоинт: автоматически вычислен batch_idx={batch_idx} для эпохи {start_epoch}")
    
    return start_epoch, global_step, batch_idx

In [ ]:
best_val_loss = float('inf')

In [ ]:
class AudioProjector(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=2048):
        super().__init__()

        self.proj = nn.Sequential(
            nn.LayerNorm(input_dim),
            nn.Linear(input_dim, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, output_dim),
            nn.LayerNorm(output_dim)
        )
        
        for layer in self.proj:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_uniform_(layer.weight)
                nn.init.zeros_(layer.bias)
    
    def forward(self, x):
        # Always return in bfloat16 for consistency with model
        return self.proj(x.float()).to(torch.bfloat16)
    
    def get_l2_norm(self):
        total_norm = 0.0
        for param in self.parameters():
            total_norm += param.data.norm(2).item() ** 2
        return total_norm ** 0.5

In [ ]:
class DatasetBlender:
    """
    🔄 Управляет плавным переходом между двумя датасетами.
    
    Поддерживает различные стратегии смешивания:
    - linear: Линейный переход от 0% до 100% второго датасета
    - cosine: Косинусный переход (более плавный)
    - exponential: Экспоненциальный переход (быстрее в конце)
    """
    
    def __init__(self, primary_data, secondary_data, transition_start_epoch, transition_end_epoch, blend_schedule="linear"):
        self.primary_data = primary_data
        self.secondary_data = secondary_data
        self.transition_start_epoch = transition_start_epoch
        self.transition_end_epoch = transition_end_epoch
        self.blend_schedule = blend_schedule
        
        print(f"🔄 DatasetBlender инициализирован:")
        print(f"   📊 Основной датасет: {len(primary_data)} примеров")
        print(f"   📊 Второй датасет: {len(secondary_data)} примеров")
        print(f"   🕐 Переход: эпохи {transition_start_epoch}-{transition_end_epoch}")
        print(f"   📈 Стратегия: {blend_schedule}")
    
    def get_blend_ratio(self, current_epoch):
        """Возвращает долю второго датасета для текущей эпохи (0.0 - 1.0)"""
        if current_epoch < self.transition_start_epoch:
            return 0.0
        elif current_epoch >= self.transition_end_epoch:
            return 1.0
        
        # Нормализованный прогресс (0.0 - 1.0)
        progress = (current_epoch - self.transition_start_epoch) / (self.transition_end_epoch - self.transition_start_epoch)
        
        if self.blend_schedule == "linear":
            return progress
        elif self.blend_schedule == "cosine":
            return (1 - np.cos(progress * np.pi)) / 2  # Плавный S-образный переход
        elif self.blend_schedule == "exponential":
            return progress ** 2  # Медленный старт, быстрый финиш
        else:
            raise ValueError(f"Неизвестная стратегия смешивания: {self.blend_schedule}")
    
    def create_blended_dataset(self, current_epoch, random_seed=42):
        """Создает смешанный датасет для текущей эпохи"""
        blend_ratio = self.get_blend_ratio(current_epoch)
        
        # Сколько примеров взять из каждого датасета
        total_size = len(self.primary_data)  # Сохраняем размер основного датасета
        secondary_count = int(total_size * blend_ratio)
        primary_count = total_size - secondary_count
        
        # Детерминированная выборка
        random_state = random.Random(random_seed)
        
        # Выбираем примеры из каждого датасета
        selected_primary = random_state.sample(self.primary_data, min(primary_count, len(self.primary_data))) if primary_count > 0 else []
        selected_secondary = random_state.sample(self.secondary_data, min(secondary_count, len(self.secondary_data))) if secondary_count > 0 else []
        
        # Объединяем и перемешиваем
        blended_data = selected_primary + selected_secondary
        random_state.shuffle(blended_data)
        
        print(f"🔄 Эпоха {current_epoch+1}: Смешивание {primary_count} основных + {secondary_count} вторичных ({blend_ratio*100:.1f}% второго)")
        
        return blended_data, blend_ratio


In [ ]:
class TrainingLogger:
    def __init__(self, experiment_name, save_dir):
        self.experiment_name = experiment_name
        self.save_dir = save_dir
        self.logs = {
            'step': [],
            'val_loss': [],
            'val_perplexity': [],
            'val_wer': [],
            'val_bleu': [],
            'val_rouge_l': []
        }
        
    def log_step(self, step, train_loss, lr_list, grad_norm=None, projector_l2_norm=None, gpu_memory_gb=None, gpu_memory_reserved_gb=None, gpu_memory_total_gb=None, memory_breakdown_mb=None, update_ratio_metrics=None):
        log_data = {
            'train/loss': float(train_loss),
            'train/projector_lr': float(lr_list[0]) if len(lr_list) > 0 else 0.0,
            'train/lora_lr': float(lr_list[1]) if len(lr_list) > 1 else 0.0,
            'train/learning_rate': float(lr_list[0]),  # Для совместимости
            'train/grad_norm': float(grad_norm) if grad_norm is not None else 0.0,
            'step': int(step)
        }
        
        if projector_l2_norm is not None:
            log_data['projector/l2_norm'] = float(projector_l2_norm)
        
        # 📊 Логируем update ratio метрики
        if update_ratio_metrics:
            if 'projector' in update_ratio_metrics:
                proj_metrics = update_ratio_metrics['projector']
                log_data['update_ratio/projector_ratio'] = proj_metrics['update_ratio']
                log_data['update_ratio/projector_status_numeric'] = proj_metrics['update_ratio_numeric']
                log_data['update_ratio/projector_grad_norm'] = proj_metrics['grad_norm']
                log_data['update_ratio/projector_weight_norm'] = proj_metrics['weight_norm']
                
            if 'lora' in update_ratio_metrics:
                lora_metrics = update_ratio_metrics['lora']
                log_data['update_ratio/lora_ratio'] = lora_metrics['update_ratio']
                log_data['update_ratio/lora_status_numeric'] = lora_metrics['update_ratio_numeric']
                log_data['update_ratio/lora_grad_norm'] = lora_metrics['grad_norm']
                log_data['update_ratio/lora_weight_norm'] = lora_metrics['weight_norm']
        
        if gpu_memory_gb is not None:
            log_data['gpu/memory_used_gb'] = float(gpu_memory_gb) if gpu_memory_gb is not None else 0.0
            log_data['gpu/memory_reserved_gb'] = float(gpu_memory_reserved_gb) if gpu_memory_reserved_gb is not None else 0.0
            log_data['gpu/memory_total_gb'] = float(gpu_memory_total_gb) if gpu_memory_total_gb is not None else 0.0
            # Fix: ensure numeric value for memory utilization
            if gpu_memory_total_gb and gpu_memory_total_gb > 0:
                log_data['gpu/memory_utilization_pct'] = float(gpu_memory_gb / gpu_memory_total_gb * 100)
            else:
                log_data['gpu/memory_utilization_pct'] = 0.0
            
        if memory_breakdown_mb:
            for k, v in memory_breakdown_mb.items():
                if k in ['grad_norm_before_clip', 'grad_norm_after_clip', 'clipping_ratio']:
                    log_data[f'gradient/{k}'] = float(v)
                elif k == 'was_clipped':
                    log_data[f'gradient/{k}'] = bool(v)
                elif k in ['prefix_l2_norm', 'projected_l2_norm', 'token_l2_norm', 'norm_ratio']:
                    log_data[f'diagnostics/{k}'] = float(v)  # 🔍 L2-нормы в отдельную категорию
                else:
                    log_data[f"memory/{k}_mb"] = float(v)

        wandb.log(log_data)
        
    def log_validation(self, step, val_metrics):
        self.logs['step'].append(int(step))
        self.logs['val_loss'].append(float(val_metrics['loss']))
        self.logs['val_perplexity'].append(float(val_metrics['perplexity']))
        self.logs['val_wer'].append(float(val_metrics['wer']))
        self.logs['val_bleu'].append(float(val_metrics['bleu']))
        self.logs['val_rouge_l'].append(float(val_metrics['rouge_l']))
        
        wandb.log({
            'val/loss': float(val_metrics['loss']),
            'val/perplexity': float(val_metrics['perplexity']),
            'val/wer': float(val_metrics['wer']),
            'val/bleu': float(val_metrics['bleu']),
            'val/rouge_l': float(val_metrics['rouge_l']),
            'step': int(step)
        })
    
    def save_logs(self):
        if len(self.logs['step']) == 0:
            return None
        df = pd.DataFrame(self.logs)
        csv_path = os.path.join(self.save_dir, 'validation_logs.csv')
        df.to_csv(csv_path, index=False)
        return df

In [ ]:
class AudioTextDataset(Dataset):
    def __init__(self, data, tokenizer, feature_extractor, zip_path=None):
        self.data = data
        self.tokenizer = tokenizer
        self.feature_extractor = feature_extractor
        self.zip_file = None
        self.zip_manifest = None
        
        if zip_path and os.path.exists(zip_path):
            try:
                self.zip_file = zipfile.ZipFile(zip_path, 'r')
                
                self.zip_manifest = {
                    p: p
                    for p in self.zip_file.namelist()
                    if p.lower().endswith(('.flac', '.wav', '.mp3'))
                }
                
                wandb.log({
                    "dataset/zip_loaded": 1.0,  # Convert bool to numeric
                    "dataset/zip_audio_files": int(len(self.zip_manifest)),
                    "dataset/total_records": int(len(self.data))
                })

            except Exception as e:
                self.zip_file = None
                wandb.log({"dataset/zip_error": str(e)})
        else:
            wandb.log({"dataset/zip_loaded": 0.0, "dataset/total_records": int(len(self.data))})
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        audio_path = item["audio_path"]
        speaker_text = item["speaker_text"]
        
        try:
            if self.zip_file and self.zip_manifest is not None:
                found_path = self.zip_manifest.get(audio_path)
                if found_path:
                    with self.zip_file.open(found_path) as audio_file:
                        audio_data = audio_file.read()
                        waveform, sr = torchaudio.load(io.BytesIO(audio_data))
                else:
                    raise FileNotFoundError(f"Файл '{audio_path}' не найден в манифесте ZIP.")
            else:
                waveform, sr = torchaudio.load(audio_path)
                
        except Exception as e:
            print(f"⚠️ Ошибка загрузки {audio_path}: {e}")
            waveform = torch.zeros(1, 16000)
            sr = 16000
        
        if sr != self.feature_extractor.sampling_rate:
            waveform = torchaudio.functional.resample(waveform, sr, self.feature_extractor.sampling_rate)
        
        if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0, keepdim=True)
        
        waveform_np = waveform.squeeze().numpy()
        waveform_mean = np.mean(waveform_np)
        waveform_std = np.std(waveform_np)
        
        if waveform_std > 1e-8:
            waveform_np = (waveform_np - waveform_mean) / waveform_std
        
        inputs = self.feature_extractor(
            waveform_np,
            sampling_rate=self.feature_extractor.sampling_rate,
            return_tensors="pt"
        )
        
        tokens = self.tokenizer(
            speaker_text,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        )
        return {
            "input_values": inputs.input_values.squeeze(0),
            "input_ids": tokens.input_ids.squeeze(0),
            "attention_mask": tokens.attention_mask.squeeze(0)
        }
    
    def __del__(self):
        if hasattr(self, 'zip_file') and self.zip_file:
            self.zip_file.close()

In [ ]:
def compress_audio_features(audio_features, compression_rate_k):
    batch_size, seq_len, hidden_dim = audio_features.shape
    
    new_seq_len = (seq_len // compression_rate_k) * compression_rate_k
    audio_features = audio_features[:, :new_seq_len, :]
    
    reshaped = audio_features.view(batch_size, new_seq_len // compression_rate_k, compression_rate_k, hidden_dim)
    compressed = reshaped.view(batch_size, new_seq_len // compression_rate_k, compression_rate_k * hidden_dim)
    
    return compressed

In [ ]:
def evaluate_with_analysis(model, projector, wav2vec2, dataloader, tokenizer, prefix_embeds, device, 
                          max_new_tokens, compression_rate_k, beam_width, temperature, top_k, top_p, repetition_penalty,
                          analyzer=None, current_step=0, run_analysis=False, analysis_config=None):
    """
    Расширенная функция валидации с интеграцией исследовательского анализа
    
    Args:
        analyzer: экземпляр AudioTextEmbeddingAnalyzer
        current_step: текущий шаг обучения
        run_analysis: запускать ли анализ на этом шаге
        analysis_config: конфигурация анализа
    """
    model.eval()
    projector.eval()
    wav2vec2.eval()
    total_loss = 0.0
    total_wer = 0.0
    total_bleu = 0.0
    total_rouge_l = 0.0
    count = 0
    examples_shown = 0
    smooth = SmoothingFunction().method1
    rouge_scorer_obj = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    
    # 🔬 Переменные для исследовательского анализа
    analysis_results = {}
    analysis_batches_collected = []
    
    with torch.no_grad():
        for batch_idx, batch in enumerate(tqdm(dataloader, desc="Validation")):
            input_ids = batch["input_ids"].to(device)

            outputs, prompt_embeds, norm_diagnostics = process_batch(
                batch, model, projector, wav2vec2, tokenizer, prefix_embeds, device, compression_rate_k,
                ctc_head=None, ctc_loss_fn=None  # Отключаем CTC во время валидации
            )
            loss = outputs.loss
            total_loss += loss.item()

            # 🔬 Собираем батчи для анализа (только первые несколько)
            if run_analysis and analyzer and len(analysis_batches_collected) < analysis_config.get('analysis_batch_size', 2):
                # Создаем копию батча для анализа
                analysis_batch = {
                    'input_values': batch['input_values'].clone(),
                    'input_ids': batch['input_ids'].clone(),
                    'step': current_step,
                    'batch_idx': batch_idx
                }
                analysis_batches_collected.append(analysis_batch)

            # Keep bfloat16 for generation consistency
            prompt_embeds = prompt_embeds.to(dtype=torch.bfloat16)

            generated_ids = model.generate(
                inputs_embeds=prompt_embeds,
                max_new_tokens=max_new_tokens,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.pad_token_id,
                num_beams=beam_width,
                temperature=temperature,
                do_sample=True,
                top_k=top_k,
                top_p=top_p,
                repetition_penalty=repetition_penalty,
                early_stopping=True
            )

            for j in range(generated_ids.size(0)):
                pred_text = tokenizer.decode(generated_ids[j], skip_special_tokens=True).strip()
                ref_text_ids = input_ids[j]
                ref_text_ids = ref_text_ids[ref_text_ids != -100]
                ref_text = tokenizer.decode(ref_text_ids, skip_special_tokens=True).strip()
                
                if ref_text and pred_text:
                    current_wer = jiwer.wer(ref_text, pred_text)
                    current_bleu = sentence_bleu([ref_text.split()], pred_text.split(), smoothing_function=smooth)
                    rouge_scores = rouge_scorer_obj.score(ref_text, pred_text)
                    
                    total_wer += current_wer
                    total_bleu += current_bleu
                    total_rouge_l += rouge_scores['rougeL'].fmeasure
                    count += 1
                    
                    if examples_shown < 3:
                        print(f"\nПример {examples_shown + 1}:")
                        print(f"Эталон: '{ref_text}'")
                        print(f"Генерация: '{pred_text}'")
                        print(f"WER: {current_wer:.3f}, BLEU: {current_bleu:.3f}")
                        examples_shown += 1
                    
    # 🔬 Запускаем исследовательский анализ
    if run_analysis and analyzer and len(analysis_batches_collected) > 0:
        print(f"\n🔬 Запуск исследовательского анализа на шаге {current_step}...")
        
        try:
            # Анализируем каждый собранный батч
            for i, analysis_batch in enumerate(analysis_batches_collected):
                sample_name = f"step_{current_step}_batch_{analysis_batch['batch_idx']}"
                
                batch_analysis = analyzer.run_comprehensive_analysis(
                    batch=analysis_batch,
                    compression_rate_k=compression_rate_k,
                    prefix_embeds=prefix_embeds,
                    sample_name=sample_name
                )
                
                analysis_results[f'batch_{i}'] = batch_analysis
                
                # Логируем результаты анализа в wandb
                wandb.log({
                    f"analysis/step_{current_step}/batch_{i}/diagonal_alignment": batch_analysis['alignment_metrics']['diagonal_alignment_score'],
                    f"analysis/step_{current_step}/batch_{i}/max_alignment": batch_analysis['alignment_metrics']['max_alignment_score'],
                    f"analysis/step_{current_step}/batch_{i}/attention_entropy": batch_analysis['alignment_metrics']['attention_entropy'],
                    f"analysis/step_{current_step}/batch_{i}/similarity_mean": batch_analysis['similarity_stats']['mean_similarity'],
                    f"analysis/step_{current_step}/batch_{i}/similarity_std": batch_analysis['similarity_stats']['std_similarity'],
                    "step": current_step
                })
                
                # 🎯 Логируем строку ближайших токенов (новая функция!)
                if 'tokens_string' in batch_analysis:
                    tokens_data = batch_analysis['tokens_string']
                    wandb.log({
                        f"analysis/step_{current_step}/batch_{i}/tokens_string_readable": tokens_data['nearest_tokens_readable'],
                        f"analysis/step_{current_step}/batch_{i}/tokens_string_avg_similarity": tokens_data['statistics']['avg_similarity'],
                        f"analysis/step_{current_step}/batch_{i}/tokens_string_length": tokens_data['statistics']['sequence_length'],
                        "step": current_step
                    })
                
                # Логируем примеры ближайших токенов
                if analysis_config.get('run_nearest_tokens_analysis', True) and 'nearest_tokens' in batch_analysis:
                    nearest_examples = batch_analysis['nearest_tokens'][:3]  # Первые 3 временных шага
                    for j, timestep_data in enumerate(nearest_examples):
                        top_token = timestep_data['nearest_tokens'][0]  # Самый близкий токен
                        wandb.log({
                            f"analysis/step_{current_step}/batch_{i}/timestep_{j}/top_token_similarity": top_token['similarity'],
                            f"analysis/step_{current_step}/batch_{i}/timestep_{j}/top_token_text": top_token['token_text'],
                            "step": current_step
                        })
            
            print(f"✅ Исследовательский анализ завершен для шага {current_step}")
            
        except Exception as e:
            print(f"⚠️ Ошибка в исследовательском анализе: {e}")
            analysis_results['error'] = str(e)
    
    # Стандартные метрики валидации
    avg_loss = total_loss / len(dataloader)
    perplexity = torch.exp(torch.tensor(avg_loss)).item()
    avg_wer = total_wer / count if count > 0 else 0.0
    avg_bleu = total_bleu / count if count > 0 else 0.0
    avg_rouge_l = total_rouge_l / count if count > 0 else 0.0
    
    print(f"\nВалидация ({count} примеров):")
    print(f"Loss: {avg_loss:.4f}, WER: {avg_wer:.4f}, BLEU: {avg_bleu:.4f}")
    
    # Возвращаем стандартные метрики + результаты анализа
    results = {
        'loss': avg_loss, 'perplexity': perplexity,
        'wer': avg_wer, 'bleu': avg_bleu, 'rouge_l': avg_rouge_l
    }
    
    if analysis_results:
        results['analysis'] = analysis_results
    
    return results

print("✅ Функция evaluate_with_analysis создана!")
print("🔬 Поддерживает интеграцию анализатора embedding'ов в процесс валидации")


In [ ]:
def evaluate_with_metrics(model, projector, wav2vec2, dataloader, tokenizer, prefix_embeds, device, max_new_tokens, compression_rate_k, beam_width, temperature, top_k, top_p, repetition_penalty):
    model.eval()
    projector.eval()
    wav2vec2.eval()
    total_loss = 0.0
    total_wer = 0.0
    total_bleu = 0.0
    total_rouge_l = 0.0
    count = 0
    examples_shown = 0
    smooth = SmoothingFunction().method1
    rouge_scorer_obj = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Validation"):
            input_ids = batch["input_ids"].to(device)

            outputs, prompt_embeds, norm_diagnostics = process_batch(
                batch, model, projector, wav2vec2, tokenizer, prefix_embeds, device, compression_rate_k,
                ctc_head=None, ctc_loss_fn=None  # Отключаем CTC во время валидации
            )
            loss = outputs.loss
            total_loss += loss.item()

            # Keep bfloat16 for generation consistency
            prompt_embeds = prompt_embeds.to(dtype=torch.bfloat16)

            generated_ids = model.generate(
                inputs_embeds=prompt_embeds,
                max_new_tokens=max_new_tokens,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.pad_token_id,
                num_beams=beam_width,
                temperature=temperature,
                do_sample=True,
                top_k=top_k,
                top_p=top_p,
                repetition_penalty=repetition_penalty,
                early_stopping=True
            )

            for j in range(generated_ids.size(0)):
                pred_text = tokenizer.decode(generated_ids[j], skip_special_tokens=True).strip()
                ref_text_ids = input_ids[j]
                ref_text_ids = ref_text_ids[ref_text_ids != -100]
                ref_text = tokenizer.decode(ref_text_ids, skip_special_tokens=True).strip()
                
                if ref_text and pred_text:
                    current_wer = jiwer.wer(ref_text, pred_text)
                    current_bleu = sentence_bleu([ref_text.split()], pred_text.split(), smoothing_function=smooth)
                    rouge_scores = rouge_scorer_obj.score(ref_text, pred_text)
                    
                    total_wer += current_wer
                    total_bleu += current_bleu
                    total_rouge_l += rouge_scores['rougeL'].fmeasure
                    count += 1
                    
                    if examples_shown < 3:
                        print(f"\nПример {examples_shown + 1}:")
                        print(f"Эталон: '{ref_text}'")
                        print(f"Генерация: '{pred_text}'")
                        print(f"WER: {current_wer:.3f}, BLEU: {current_bleu:.3f}")
                        examples_shown += 1
                    
    avg_loss = total_loss / len(dataloader)
    perplexity = torch.exp(torch.tensor(avg_loss)).item()
    avg_wer = total_wer / count if count > 0 else 0.0
    avg_bleu = total_bleu / count if count > 0 else 0.0
    avg_rouge_l = total_rouge_l / count if count > 0 else 0.0
    
    print(f"\nВалидация ({count} примеров):")
    print(f"Loss: {avg_loss:.4f}, WER: {avg_wer:.4f}, BLEU: {avg_bleu:.4f}")
    
    return {
        'loss': avg_loss, 'perplexity': perplexity,
        'wer': avg_wer, 'bleu': avg_bleu, 'rouge_l': avg_rouge_l
    }

In [ ]:
device = torch.device("cuda")
model_id = "google/gemma-3-4b-pt"
audio_model_name = "facebook/wav2vec2-xls-r-300m"

batch_size = 4
num_epochs = 10
projector_learning_rate = 5e-4  # 🔧 Увеличен warmup LR
lora_learning_rate = 2e-4
weight_decay = 0.1  # 🔧 Увеличено для стабилизации обучения
max_grad_norm = 10.0
gradient_accumulation_steps = 8  # 🔧 Увеличен до 8, чтобы эффективный batch size = 4*8 = 32
save_every_steps = 2000
save_latest_every_steps = 50
max_new_tokens = 70
compression_rate_k = 3
beam_width = 15
temperature = 0.6
top_k = 50
top_p = 0.9
repetition_penalty = 1.5
val_subset_size = 15
use_8bit_optimizer = True

# 🔬 Новые параметры для исследовательского анализа
enable_embedding_analysis = True   # Включить анализ embedding'ов
analysis_frequency = 500          # Каждые N шагов (вместе с основной валидацией)
save_plots_to_wandb = True        # Сохранять графики в Weights & Biases вместо локальной папки
run_tsne_analysis = True          # Запускать t-SNE анализ
run_similarity_heatmap = True     # Создавать матрицы косинусной близости
run_nearest_tokens_analysis = True # Поиск ближайших токенов
analysis_batch_size = 2           # Размер батча для анализа (меньше для экономии памяти)

# 📊 Параметры мониторинга Update Ratio для контроля эффективности обучения
enable_update_ratio_monitoring = True  # Включить мониторинг update ratio
update_ratio_logging_frequency = 10     # Каждые N шагов (1 = каждый шаг)
show_update_ratio_in_progress = True   # Показывать в прогресс-баре с эмодзи
update_ratio_alert_threshold = 1e-5    # Порог для предупреждений о микроскопических обновлениях

# 🔄 Новые параметры для плавного перехода между датасетами
enable_dataset_blending = False  # Включить смешивание датасетов
transition_start_epoch = 8      # Эпоха начала перехода (с 8-й эпохи)
transition_end_epoch = 10       # Эпоха завершения перехода (10-я эпоха = 100% второй датасет)
blend_schedule = "linear"       # Тип перехода: "linear", "cosine", "exponential"

# Пути к датасетам
primary_jsonl_path = "transcripts.jsonl"     # Основной датасет
primary_zip_path = "LibriSpeech.zip"
secondary_jsonl_path = "transcripts_v2.jsonl"  # Второй датасет (можно заменить)
secondary_zip_path = "LibriSpeech_v2.zip"      # Второй ZIP (можно заменить)

input_dim = 1024
output_dim = 2560

experiment_name = f"audio_projector_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
checkpoint_dir = "/home/jovyan/persistent_volume/"
resume_training = True
checkpoint_path = "/home/jovyan/persistent_volume/latest_checkpoint_bs4_epoch_1_step_2000.pt"
skip_validation = False
interactive_mode = True

wandb.init(
    project="audio-projector",
    name=experiment_name,
    config={
        "batch_size": batch_size,
        "num_epochs": num_epochs,
        "projector_learning_rate": projector_learning_rate,
        "lora_learning_rate": lora_learning_rate,
        "weight_decay": weight_decay,
        "max_grad_norm": max_grad_norm,
        "gradient_accumulation_steps": gradient_accumulation_steps,
        "max_new_tokens": max_new_tokens,
        "compression_rate_k": compression_rate_k,
        "beam_width": beam_width,
        "temperature": temperature,
        "top_k": top_k,
        "top_p": top_p,
        "repetition_penalty": repetition_penalty,
        "val_subset_size": val_subset_size,
        "input_dim": input_dim,
        "output_dim": output_dim,
        "model_id": model_id,
        "audio_model_name": audio_model_name,
        "resume_training": resume_training,
        "z_normalization": True,
        "projector_hidden_dim": 2048,
        "activation": "GELU",
        "scheduler_type": "CosineAnnealingWarmRestarts",
        "use_8bit_optimizer": use_8bit_optimizer,
        "optimizer_type": "AdamW8bit" if use_8bit_optimizer else "AdamW",
        "mse_loss_removed": "MSE loss disabled due to alignment issues",
        "enable_dataset_blending": enable_dataset_blending,
        "transition_start_epoch": transition_start_epoch,
        "transition_end_epoch": transition_end_epoch,
        "blend_schedule": blend_schedule,
        "primary_jsonl_path": primary_jsonl_path,
        "secondary_jsonl_path": secondary_jsonl_path,
        "lora_config": {
            "r": 64,
            "lora_alpha": 128,
            "target_modules": ["k_proj", "v_proj", "o_proj", "gate_proj", "up_proj"],
            "lora_dropout": 0.05
        },
        "analysis_config": {
            "enable_embedding_analysis": enable_embedding_analysis,
            "analysis_frequency": analysis_frequency,
            "save_plots_to_wandb": save_plots_to_wandb,
            "run_tsne_analysis": run_tsne_analysis,
            "run_similarity_heatmap": run_similarity_heatmap,
            "run_nearest_tokens_analysis": run_nearest_tokens_analysis,
            "analysis_batch_size": analysis_batch_size
        },
        "update_ratio_config": {
            "enable_update_ratio_monitoring": enable_update_ratio_monitoring,
            "update_ratio_logging_frequency": update_ratio_logging_frequency,
            "show_update_ratio_in_progress": show_update_ratio_in_progress,
            "update_ratio_alert_threshold": update_ratio_alert_threshold
        }
    }
)

best_val_loss = float('inf')
best_checkpoint_path = None
latest_checkpoint_path = None

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True
)

hf_token = os.getenv('HF_TOKEN')
if hf_token:
    login(token=hf_token)


In [ ]:
notebook_login()

tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

multi_cfg = AutoConfig.from_pretrained(model_id, token=hf_token)

text_cfg_dict = multi_cfg.text_config.to_dict()
text_cfg_dict["vocab_size"] = 262208
text_cfg_dict.update({"bos_token_id": tokenizer.bos_token_id, "eos_token_id": tokenizer.eos_token_id, "pad_token_id": tokenizer.pad_token_id})

text_cfg = Gemma3TextConfig(**text_cfg_dict)
gemma_model = Gemma3ForCausalLM.from_pretrained(
    model_id,
    config=text_cfg,
    torch_dtype=torch.bfloat16,
    quantization_config=quantization_config,
    device_map="cuda",
    token=hf_token
)

gemma_model.gradient_checkpointing_enable()
gemma_model = prepare_model_for_kbit_training(gemma_model)

lora_config = LoraConfig(
    r=64,
    lora_alpha=128,
    target_modules=["k_proj", "v_proj", "o_proj", "gate_proj", "up_proj"],
    lora_dropout=0.2,
    bias="none",
    init_lora_weights=False,
    task_type="CAUSAL_LM"
)

gemma_model = get_peft_model(gemma_model, lora_config)

# Cast lm_head to bfloat16 for consistency with the rest of the model
if hasattr(gemma_model, 'lm_head'):
    gemma_model.lm_head = gemma_model.lm_head.to(torch.bfloat16)
elif hasattr(gemma_model, 'base_model') and hasattr(gemma_model.base_model, 'lm_head'):
    gemma_model.base_model.lm_head = gemma_model.base_model.lm_head.to(torch.bfloat16)

gemma_model.eval()

In [ ]:
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(audio_model_name)
wav2vec2 = Wav2Vec2Model.from_pretrained(audio_model_name)
wav2vec2 = wav2vec2.to(torch.bfloat16).to(device)
wav2vec2.eval()
for param in wav2vec2.parameters():
    param.requires_grad = False

In [ ]:
# 🔧 Пункт 5: Диагностика градиентов MLP параметров для отладки обучения
def check_mlp_gradients(projector, prefix="projector"):
    """Проверяет градиенты MLP параметров для диагностики dead ReLU и малых α"""
    gradient_norms = {}
    
    for name, param in projector.named_parameters():
        if param.grad is not None:
            grad_norm = param.grad.norm().item()
            gradient_norms[f"{prefix}/{name}"] = grad_norm
            
            # Проверка на очень маленькие градиенты (возможный dead ReLU)
            if grad_norm < 1e-8:
                print(f"⚠️ Очень маленький градиент в {name}: {grad_norm:.2e}")
        else:
            gradient_norms[f"{prefix}/{name}"] = 0.0
            print(f"⚠️ Отсутствует градиент в {name}")
    
    return gradient_norms

print("🔧 Функция диагностики градиентов MLP добавлена")


In [ ]:
projector_input_dim = input_dim * compression_rate_k
projector_hidden_dim = 2048
projector = AudioProjector(projector_input_dim, output_dim, hidden_dim=projector_hidden_dim).to(device).float()

# 🔤 Обновляем параметры оптимизации включая CTC головку
params_to_optimize = [
    {"params": projector.parameters(), "lr": projector_learning_rate},
    {"params": gemma_model.parameters(), "lr": lora_learning_rate},
    {"params": ctc_head.parameters(), "lr": projector_learning_rate}  # CTC на том же LR что и проектор
]

# Используем 8-битный оптимизатор для экономии памяти (~75% снижение использования памяти)
if use_8bit_optimizer:
    optimizer = bnb.optim.AdamW8bit(
        params_to_optimize,
        weight_decay=weight_decay,
        betas=(0.9, 0.999),
        eps=1e-8
    )
    print("✅ Используется 8-битный AdamW оптимизатор (экономия памяти ~75%)")
else:
    optimizer = optim.AdamW(
        params_to_optimize,
        weight_decay=weight_decay,
        betas=(0.9, 0.999),
        eps=1e-8
    )
    print("⚠️ Используется обычный AdamW оптимизатор")

scaler = GradScaler()
loss_fn = nn.CrossEntropyLoss(ignore_index=-100)

prefix = "Transcribe speech to text."
prefix_ids = tokenizer(prefix, return_tensors="pt").input_ids.to(device)
with torch.no_grad():
    prefix_embeds = gemma_model.get_input_embeddings()(prefix_ids).to(dtype=torch.bfloat16)

In [ ]:
# 🔧 Пункт 3: CTC Loss для лучшего выравнивания аудио-текста
import string

class CTCHead(nn.Module):
    """Легковесная CTC головка для фонемно-осознанного энкодера"""
    
    def __init__(self, input_dim, vocab_size=29):  # 26 букв + пробел + апостроф + blank
        super().__init__()
        self.linear = nn.Linear(input_dim, vocab_size)
        self.vocab_size = vocab_size
        
    def forward(self, wav2vec_outputs):
        # wav2vec_outputs: [batch, seq_len, hidden_dim]
        logits = self.linear(wav2vec_outputs)  # [batch, seq_len, vocab_size]
        return torch.log_softmax(logits, dim=-1)

# CTC словарь (буквы + пробел + апостроф + blank)
CTC_VOCAB = list(string.ascii_lowercase) + [' ', "'", '<blank>']  # 26 + 1 + 1 + 1 = 29
CTC_CHAR_TO_IDX = {char: idx for idx, char in enumerate(CTC_VOCAB)}
CTC_BLANK_IDX = CTC_CHAR_TO_IDX['<blank>']

def text_to_ctc_targets(text_batch, char_to_idx=CTC_CHAR_TO_IDX):
    """Конвертирует батч текстов в CTC целевые последовательности"""
    ctc_targets = []
    target_lengths = []
    
    for text in text_batch:
        # Очищаем текст: только буквы, пробелы и апострофы
        cleaned = ''.join(c.lower() for c in text if c.lower() in char_to_idx and c != '<blank>')
        
        # Конвертируем в индексы
        target = [char_to_idx[c] for c in cleaned if c in char_to_idx]
        
        ctc_targets.extend(target)
        target_lengths.append(len(target))
    
    return torch.tensor(ctc_targets, dtype=torch.long), torch.tensor(target_lengths, dtype=torch.long)

# Создаем CTC головку
ctc_head = CTCHead(input_dim=1024, vocab_size=len(CTC_VOCAB)).to(device).float()  # wav2vec2 выходная размерность
ctc_loss_fn = nn.CTCLoss(blank=CTC_BLANK_IDX, zero_infinity=True)

print(f"🔤 CTC словарь ({len(CTC_VOCAB)} символов): {CTC_VOCAB}")
print(f"🔤 CTC blank индекс: {CTC_BLANK_IDX}")
print(f"🔤 CTC головка создана: {input_dim} → {len(CTC_VOCAB)}")

# Добавляем CTC параметры к оптимизатору
params_to_optimize_with_ctc = [
    {"params": projector.parameters(), "lr": projector_learning_rate},
    {"params": gemma_model.parameters(), "lr": lora_learning_rate},
    {"params": ctc_head.parameters(), "lr": projector_learning_rate}  # CTC на том же LR что и проектор
]

In [ ]:
# 🔬 Инициализация исследовательского анализатора
embedding_analyzer = None

if enable_embedding_analysis:
    try:
        print("🔬 Создание анализатора embedding'ов...")
        embedding_analyzer = AudioTextEmbeddingAnalyzer(
            model=gemma_model,
            projector=projector,
            wav2vec2=wav2vec2,
            tokenizer=tokenizer,
            device=device,
            save_to_wandb=save_plots_to_wandb
        )
        print(f"✅ Анализатор создан! {'Графики будут отправляться в Weights & Biases' if save_plots_to_wandb else 'Графики будут отображаться локально'}")
        
        wandb.log({
            "analysis/analyzer_initialized": True,
            "analysis/vocab_size_analyzed": len(embedding_analyzer.vocab_embeddings),
            "analysis/save_to_wandb": save_plots_to_wandb
        })
        
    except Exception as e:
        print(f"⚠️ Ошибка при создании анализатора: {e}")
        print("🔧 Отключаем анализ embedding'ов")
        enable_embedding_analysis = False
        embedding_analyzer = None
        
        wandb.log({
            "analysis/analyzer_error": str(e),
            "analysis/analyzer_disabled": True
        })
else:
    print("🔬 Анализ embedding'ов отключен")
    wandb.log({"analysis/analyzer_disabled": True})

print(f"🔬 Статус анализатора: {'✅ Активен' if embedding_analyzer else '❌ Отключен'}")

# Создаем конфигурацию анализа для передачи в функции
analysis_config = {
    'enable_embedding_analysis': enable_embedding_analysis,
    'analysis_frequency': analysis_frequency,
    'save_plots_to_wandb': save_plots_to_wandb,
    'run_tsne_analysis': run_tsne_analysis,
    'run_similarity_heatmap': run_similarity_heatmap,
    'run_nearest_tokens_analysis': run_nearest_tokens_analysis,
    'analysis_batch_size': analysis_batch_size
}


In [ ]:
# 🔄 Загрузка данных с поддержкой плавного перехода между датасетами
def load_dataset_data(jsonl_path, zip_path=None):
    """Загружает и нормализует данные из JSONL файла"""
    try:
        with open(jsonl_path, "r", encoding="utf-8") as f:
            raw_data = [json.loads(line) for line in f]
        
        # Нормализация данных
        normalized_data = []
        for item in raw_data:
            normalized_item = {
                "audio_path": item.get("audio_filepath", ""),
                "speaker_text": item.get("text", ""),
                "language": item.get("language", "en"),
                "source": item.get("source", "unknown")
            }
            normalized_data.append(normalized_item)
        
        print(f"📊 Загружено {len(normalized_data)} примеров из {jsonl_path}")
        return normalized_data
    
    except FileNotFoundError:
        print(f"⚠️ Файл {jsonl_path} не найден, возвращаем пустой список")
        return []

# Загружаем основной датасет
primary_data = load_dataset_data(primary_jsonl_path, primary_zip_path)

# Загружаем второй датасет (если включено смешивание)
secondary_data = []
if enable_dataset_blending:
    secondary_data = load_dataset_data(secondary_jsonl_path, secondary_zip_path)
    if len(secondary_data) == 0:
        print(f"⚠️ Второй датасет пуст, отключаем смешивание")
        enable_dataset_blending = False

# Объединяем для train/val split
if enable_dataset_blending:
    all_data = primary_data + secondary_data  # Для создания единого val_data
    print(f"📊 Объединено {len(primary_data)} + {len(secondary_data)} = {len(all_data)} примеров")
else:
    all_data = primary_data
    print(f"📊 Используется только основной датасет: {len(all_data)} примеров")

# Создаем единый val_data из всех доступных данных
total_records = len(all_data)
_, val_data = train_test_split(all_data, test_size=0.1, random_state=42)

# 🔄 Инициализируем DatasetBlender если включено смешивание
dataset_blender = None
if enable_dataset_blending:
    # Разделяем primary_data на train/val с тем же random_state
    primary_train_data, _ = train_test_split(primary_data, test_size=0.1, random_state=42)
    secondary_train_data, _ = train_test_split(secondary_data, test_size=0.1, random_state=42)
    
    dataset_blender = DatasetBlender(
        primary_data=primary_train_data,
        secondary_data=secondary_train_data,
        transition_start_epoch=transition_start_epoch,
        transition_end_epoch=transition_end_epoch,
        blend_schedule=blend_schedule
    )
    train_data = primary_train_data  # Начинаем с основного датасета
else:
    # Обычный режим без смешивания
    train_data, _ = train_test_split(all_data, test_size=0.1, random_state=42)

val_subset_data = random.sample(val_data, min(val_subset_size, len(val_data)))

print(f"📊 Data: {len(train_data)} train, {len(val_subset_data)} val")

In [ ]:
# ИСПРАВЛЕНО: пропускаем данные на уровне JSON, а не батчей!
# Это будет установлено после загрузки чекпоинта
skip_samples_from_checkpoint = 0

train_dataset = AudioTextDataset(train_data, tokenizer, feature_extractor, zip_path=primary_zip_path)
val_dataset = AudioTextDataset(val_subset_data, tokenizer, feature_extractor, zip_path=primary_zip_path)

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,  # Будет изменен на False при возобновлении
    collate_fn=collate_fn,
    num_workers=0,
    pin_memory=False
)
val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=0,
    pin_memory=False
)

print(f"🔧 DataLoaders: {len(train_loader)} train, {len(val_loader)} val batches")


In [ ]:
wav2vec2 = wav2vec2.to(device)
projector = projector.to(device)
gemma_model = gemma_model.to(device)

total_steps = num_epochs * len(train_loader) // gradient_accumulation_steps

def calculate_restart_period(base_examples, batch_size, grad_accum_steps):
    """
    Рассчитывает период рестарта в шагах оптимизации.
    
    Args:
        base_examples: Базовое количество примеров до рестарта
        batch_size: Текущий физический размер батча
        grad_accum_steps: Количество шагов градиентного накопления
    
    Returns:
        int: Количество шагов оптимизации до рестарта
    """
    actual_batch_size = batch_size * grad_accum_steps
    restart_steps = max(1, base_examples // actual_batch_size)
    return restart_steps

base_examples = 30000
adaptive_restart_period = calculate_restart_period(
    base_examples=base_examples,
    batch_size=batch_size,
    grad_accum_steps=gradient_accumulation_steps
)

scheduler = CosineAnnealingWarmRestarts(
    optimizer,
    T_0=adaptive_restart_period,
    T_mult=1,
    eta_min=1e-6  # 🔧 Cosine decay до 1e-6
)

print(f"🔧 Training: {total_steps} steps, LR({projector_learning_rate}/{lora_learning_rate}), GradAcc({gradient_accumulation_steps})")
print(f"🔄 Scheduler: CosineAnnealingWarmRestarts с периодом {adaptive_restart_period} шагов")

os.makedirs(checkpoint_dir, exist_ok=True)
logger = TrainingLogger(experiment_name, checkpoint_dir)

In [ ]:
def get_gpu_memory_stats(device):
    """Получить статистику использования GPU памяти"""
    if not torch.cuda.is_available():
        return None, None, None
    
    try:
        allocated = torch.cuda.memory_allocated(device) / 1024**3  # GB
        reserved = torch.cuda.memory_reserved(device) / 1024**3   # GB
        
        # Попытаемся получить общий объем GPU памяти
        gpu_properties = torch.cuda.get_device_properties(device)
        total_memory = gpu_properties.total_memory / 1024**3  # GB
        
        return allocated, reserved, total_memory
    except Exception as e:
        print(f"⚠️ Ошибка получения GPU статистики: {e}")
        return None, None, None

def light_gpu_cleanup(device):
    """
    Легкая очистка GPU памяти, которая не удаляет переменные,
    а только очищает кэш и собирает мусор.
    """
    import gc
    import torch
    if not torch.cuda.is_available():
        print("🧹 GPU недоступен, выполняется только сборка мусора.")
        gc.collect()
        return

    # Синхронизация для завершения всех текущих операций
    torch.cuda.synchronize(device)
    
    # Сбор мусора Python
    gc.collect()
    
    # Очистка кэша PyTorch
    torch.cuda.empty_cache()
    
    # Дополнительная очистка для межпроцессного взаимодействия
    torch.cuda.ipc_collect()
    
    # Сброс статистики для более точного мониторинга
    torch.cuda.reset_peak_memory_stats(device)
    torch.cuda.reset_accumulated_memory_stats(device)
    
    print(f"🧹 Легкая очистка GPU завершена.")

In [ ]:
def get_gpu_memory_stats(device):
    """Получить статистику использования GPU памяти"""
    if not torch.cuda.is_available():
        return None, None, None
    
    try:
        allocated = torch.cuda.memory_allocated(device) / 1024**3  # GB
        reserved = torch.cuda.memory_reserved(device) / 1024**3   # GB
        
        # Попытаемся получить общий объем GPU памяти
        gpu_properties = torch.cuda.get_device_properties(device)
        total_memory = gpu_properties.total_memory / 1024**3  # GB
        
        return allocated, reserved, total_memory
    except Exception as e:
        print(f"⚠️ Ошибка получения GPU статистики: {e}")
        return None, None, None

def get_model_memory_footprint(model: nn.Module, trainable_only: bool = False) -> float:
    """Calculates the memory footprint of a model's parameters in megabytes."""
    total_bytes = 0
    for param in model.parameters():
        if trainable_only and not param.requires_grad:
            continue
        total_bytes += param.nelement() * param.element_size()
    return total_bytes / 1024**2

def light_gpu_cleanup(device):
    """
    Легкая очистка GPU памяти, которая не удаляет переменные,
    а только очищает кэш и собирает мусор.
    """
    import gc
    import torch
    if not torch.cuda.is_available():
        print("🧹 GPU недоступен, выполняется только сборка мусора.")
        gc.collect()
        return

    # Синхронизация для завершения всех текущих операций
    torch.cuda.synchronize(device)
    
    # Сбор мусора Python
    gc.collect()
    
    # Очистка кэша PyTorch
    torch.cuda.empty_cache()
    
    # Дополнительная очистка для межпроцессного взаимодействия
    torch.cuda.ipc_collect()
    
    # Сброс статистики для более точного мониторинга
    torch.cuda.reset_peak_memory_stats(device)
    torch.cuda.reset_accumulated_memory_stats(device)
    
    print(f"🧹 Легкая очистка GPU завершена.")


In [ ]:
# 📊 Функция анализа Update Ratio для диагностики силы градиентов
def analyze_gradients_and_weights(model, prefix="projector"):
    """
    🔬 Анализирует отношение силы градиентов к весам модели (update ratio)
    
    Update Ratio = grad_norm / (weight_norm + 1e-8)
    
    Хорошие значения:
    - 1e-4 - 1e-3: рабочие значения  
    - < 1e-5: микроскопические обновления (возможная стагнация)
    - > 1e-2: слишком агрессивные обновления
    
    Args:
        model: модель для анализа (projector)
        prefix: префикс для логирования
        
    Returns:
        dict: метрики градиентов и весов
    """
    metrics = {}
    
    with torch.no_grad():
        # 📏 Вычисляем норму градиентов
        total_grad_norm = 0.0
        total_weight_norm = 0.0
        grad_count = 0
        weight_count = 0
        
        # Собираем статистики по всем параметрам
        layer_metrics = {}
        
        for name, param in model.named_parameters():
            if param.requires_grad:
                # Норма весов
                weight_norm = param.data.norm(2).item()
                total_weight_norm += weight_norm ** 2
                weight_count += 1
                
                # Норма градиентов (если есть)
                if param.grad is not None:
                    grad_norm = param.grad.data.norm(2).item()
                    total_grad_norm += grad_norm ** 2
                    grad_count += 1
                    
                    # Update ratio для конкретного слоя
                    layer_update_ratio = grad_norm / (weight_norm + 1e-8)
                    
                    layer_metrics[name] = {
                        'weight_norm': weight_norm,
                        'grad_norm': grad_norm,
                        'update_ratio': layer_update_ratio
                    }
                else:
                    layer_metrics[name] = {
                        'weight_norm': weight_norm,
                        'grad_norm': 0.0,
                        'update_ratio': 0.0
                    }
        
        # Общие метрики
        total_grad_norm = total_grad_norm ** 0.5
        total_weight_norm = total_weight_norm ** 0.5
        
        # 🎯 ГЛАВНАЯ МЕТРИКА: Update Ratio
        update_ratio = total_grad_norm / (total_weight_norm + 1e-8)
        
        # Классификация update ratio
        if update_ratio < 1e-5:
            ratio_status = "МИКРОСКОПИЧЕСКИЙ"
            ratio_emoji = "🔴"
        elif update_ratio < 1e-4:
            ratio_status = "МАЛЫЙ"
            ratio_emoji = "🟠"
        elif update_ratio <= 1e-3:
            ratio_status = "ОПТИМАЛЬНЫЙ"
            ratio_emoji = "🟢"
        elif update_ratio <= 1e-2:
            ratio_status = "ВЫСОКИЙ"
            ratio_emoji = "🟡"
        else:
            ratio_status = "СЛИШКОМ_ВЫСОКИЙ"
            ratio_emoji = "🔴"
        
        metrics = {
            f'{prefix}/total_grad_norm': total_grad_norm,
            f'{prefix}/total_weight_norm': total_weight_norm,
            f'{prefix}/update_ratio': update_ratio,
            f'{prefix}/update_ratio_status': ratio_status,
            f'{prefix}/grad_param_count': grad_count,
            f'{prefix}/weight_param_count': weight_count
        }
        
        # 📊 Статистики по слоям
        if layer_metrics:
            # Средние значения по слоям
            avg_layer_update_ratio = sum(m['update_ratio'] for m in layer_metrics.values()) / len(layer_metrics)
            max_layer_update_ratio = max(m['update_ratio'] for m in layer_metrics.values())
            min_layer_update_ratio = min(m['update_ratio'] for m in layer_metrics.values())
            
            metrics.update({
                f'{prefix}/avg_layer_update_ratio': avg_layer_update_ratio,
                f'{prefix}/max_layer_update_ratio': max_layer_update_ratio,
                f'{prefix}/min_layer_update_ratio': min_layer_update_ratio,
            })
            
            # Логируем детали лучших/худших слоев
            sorted_layers = sorted(layer_metrics.items(), key=lambda x: x[1]['update_ratio'], reverse=True)
            
            # Топ-3 слоя с наибольшим update ratio
            for i, (layer_name, layer_data) in enumerate(sorted_layers[:3]):
                metrics[f'{prefix}/top_{i+1}_layer_name'] = layer_name.replace('.', '_')  # wandb не любит точки
                metrics[f'{prefix}/top_{i+1}_layer_update_ratio'] = layer_data['update_ratio']
    
    # 🖨️ Красивый вывод в консоль
    print(f"\n{ratio_emoji} UPDATE RATIO АНАЛИЗ {ratio_emoji}")
    print(f"📊 Общий Update Ratio: {update_ratio:.2e} ({ratio_status})")
    print(f"📏 Норма градиентов: {total_grad_norm:.2e}")
    print(f"⚖️  Норма весов: {total_weight_norm:.2e}")
    print(f"🎯 Статус: {ratio_emoji} {ratio_status}")
    
    if layer_metrics:
        print(f"📋 По слоям: avg={avg_layer_update_ratio:.2e}, max={max_layer_update_ratio:.2e}, min={min_layer_update_ratio:.2e}")
        
        # Показываем проблемные слои
        problematic_layers = [(name, data) for name, data in layer_metrics.items() 
                            if data['update_ratio'] < 1e-5 or data['update_ratio'] > 1e-2]
        if problematic_layers:
            print("⚠️ Проблемные слои:")
            for name, data in problematic_layers:
                status = "МИКРО" if data['update_ratio'] < 1e-5 else "ГИГАНТ"
                print(f"   {status}: {name} = {data['update_ratio']:.2e}")
    
    return metrics

print("✅ Функция analyze_gradients_and_weights создана!")
print("📊 Анализирует update_ratio = grad_norm / (weight_norm + 1e-8)")
print("🎯 Оптимальные значения: 1e-4 - 1e-3")
print("🔴 Проблемы: < 1e-5 (стагнация) или > 1e-2 (нестабильность)")


In [ ]:
def find_latest_checkpoint(checkpoint_dir):
    pattern = os.path.join(checkpoint_dir, "latest_checkpoint_bs*_epoch*_step*.pt")    
    checkpoints = glob.glob(pattern) 
    return max(checkpoints, key=os.path.getctime) if checkpoints else None

def find_best_checkpoint(checkpoint_dir):
    pattern = os.path.join(checkpoint_dir, "best_checkpoint_bs*_step*.pt")
    checkpoints = glob.glob(pattern)
    return checkpoints[0] if checkpoints else None

def save_checkpoint(step, epoch, batch_idx=0, is_best=False):
    global best_checkpoint_path
    
    checkpoint_data = {
        'step': step,
        'epoch': epoch,
        'batch_idx': batch_idx,
        'projector_state_dict': projector.state_dict(),
        'lora_state_dict': gemma_model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'best_val_loss': best_val_loss,
        'config': {
            'projector_learning_rate': projector_learning_rate,
            'lora_learning_rate': lora_learning_rate,
            'weight_decay': weight_decay,
            'max_grad_norm': max_grad_norm,
            'batch_size': batch_size,
            'compression_rate_k': compression_rate_k,
            'input_dim': input_dim,
            'output_dim': output_dim,
            'experiment_name': experiment_name,
            'lora_config': {
                'r': 64,
                'lora_alpha': 128,
                'target_modules': ["k_proj", "v_proj", "o_proj", "gate_proj", "up_proj"],
                'lora_dropout': 0.05
            }
        }
    }
    
    if is_best:
        if best_checkpoint_path and os.path.exists(best_checkpoint_path):
            os.remove(best_checkpoint_path)
        
        best_checkpoint_path = os.path.join(checkpoint_dir, f"best_checkpoint_bs{batch_size}_step_{step}.pt")
        torch.save(checkpoint_data, best_checkpoint_path)
        print_ephemeral(f"🏆 Лучший чекпоинт сохранен: best_checkpoint_bs{batch_size}_step_{step}.pt")
    else:
        checkpoint_path = os.path.join(checkpoint_dir, f"checkpoint_bs{batch_size}_step_{step}.pt")
        torch.save(checkpoint_data, checkpoint_path)
        print_ephemeral(f"💾 Чекпоинт сохранен: checkpoint_bs{batch_size}_step_{step}.pt")

def save_latest_checkpoint(step, epoch, batch_idx=0):
    global latest_checkpoint_path
    
    checkpoint_data = {
        'step': step,
        'epoch': epoch,
        'batch_idx': batch_idx,
        'projector_state_dict': projector.state_dict(),
        'lora_state_dict': gemma_model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'best_val_loss': best_val_loss,
        'config': {
            'projector_learning_rate': projector_learning_rate,
            'lora_learning_rate': lora_learning_rate,
            'weight_decay': weight_decay,
            'max_grad_norm': max_grad_norm,
            'batch_size': batch_size,
            'compression_rate_k': compression_rate_k,
            'input_dim': input_dim,
            'output_dim': output_dim,
            'experiment_name': experiment_name,
            'lora_config': {
                'r': 64,
                'lora_alpha': 128,
                'target_modules': ["k_proj", "v_proj", "o_proj", "gate_proj", "up_proj"],
                'lora_dropout': 0.05
            }
        }
    }
    
    if latest_checkpoint_path and os.path.exists(latest_checkpoint_path):
        os.remove(latest_checkpoint_path)
    
    latest_checkpoint_path = os.path.join(checkpoint_dir, f"latest_checkpoint_bs{batch_size}_epoch_{epoch}_step_{step}.pt")
    torch.save(checkpoint_data, latest_checkpoint_path)
    
    print_ephemeral(f"📄 Последний чекпоинт: bs{batch_size}_epoch_{epoch}_step_{step}")

def print_ephemeral(message):
    """Печатает сообщение которое заменяется следующим принтом"""
    print(f"\r{' ' * 120}\r{message}", end="", flush=True)

def print_vanishing(message):
    print(f"\r{message}", end="", flush=True)

def check_user_input():
    global skip_validation
    
    try:
        import sys
        import select
        if sys.stdin in select.select([sys.stdin], [], [], 0)[0]:
            user_input = sys.stdin.readline().strip().lower()
            
            if user_input == 's':
                skip_validation = True
                print("\rПропуск валидации на следующем шаге", end="", flush=True)
    except:
        pass

In [ ]:
start_epoch = 0
global_step = 0
batch_idx = 0

if resume_training:
    if os.path.exists(checkpoint_path):
        checkpoint_epoch, global_step, batch_idx = load_checkpoint(checkpoint_path, projector, gemma_model, optimizer, scheduler, device, batch_size)
        start_epoch = checkpoint_epoch - 1
        
        # ИСПРАВЛЕННАЯ ЛОГИКА: детерминированное перемешивание + правильный пропуск
        print(f"🔄 Возобновление с эпохи {start_epoch + 1}, шага {global_step}, batch_idx {batch_idx}")
        
        # Создаем детерминированные индексы для эпохи возобновления
        random_state = random.Random(start_epoch * 12345)  # Фиксированный seed для эпохи
        shuffled_indices = list(range(len(train_data)))
        random_state.shuffle(shuffled_indices)
        
        # Вычисляем индекс в датасете, откуда продолжать
        batches_to_skip = batch_idx
        samples_to_skip = batches_to_skip * batch_size
        
        if samples_to_skip < len(shuffled_indices):
            # Берем оставшиеся индексы (НЕ теряем пропущенные данные!)
            remaining_indices = shuffled_indices[samples_to_skip:]
            remaining_train_data = [train_data[i] for i in remaining_indices]
            
            print(f"⚡ Детерминированно перемешано {len(train_data)} примеров")
            print(f"📊 Пропускаем первые {samples_to_skip} индексов, осталось: {len(remaining_train_data)} примеров")
            
            # Пересоздаем датасет с оставшимися данными по правильным индексам
            train_dataset = AudioTextDataset(remaining_train_data, tokenizer, feature_extractor, zip_path=primary_zip_path)
            train_loader = DataLoader(
                train_dataset,
                batch_size=batch_size,
                shuffle=False,  # НЕ перемешиваем - индексы уже детерминированно перемешаны
                collate_fn=collate_fn,
                num_workers=0,
                pin_memory=False
            )
            print(f"🔧 Обновлен DataLoader: {len(train_loader)} батчей")
        else:
            print(f"⚠️ Нужно пропустить {samples_to_skip} примеров, но в эпохе только {len(shuffled_indices)}")
            print("🔄 Переходим к следующей эпохе")
            start_epoch += 1
            batch_idx = 0
    else:
        resume_training = False
else:
    batch_idx = 0

print(f"🚀 Audio Projector | {wandb.run.project}/{wandb.run.name} | {'Resume' if resume_training else 'New'}")

In [ ]:
for epoch in range(start_epoch, num_epochs):
    epoch_header = f"\n{'='*50}\n"
    epoch_header += f"🔄 ЭПОХА {epoch+1}/{num_epochs}\n"
    epoch_header += f"{'='*50}"
    print_vanishing(epoch_header)
    
    projector.train()
    wav2vec2.eval()
    gemma_model.eval()
    
    is_resumed_epoch = resume_training and epoch == start_epoch
    
    # 🔄 Создание датасета с поддержкой плавного перехода
    if is_resumed_epoch:
        print(f"🔄 Эпоха {epoch+1}: продолжение с предварительно созданным DataLoader ({len(train_loader)} батчей)")
        print(f"📊 Начинаем с batch_idx={batch_idx}, global_step={global_step}")
    elif not is_resumed_epoch:
        if dataset_blender is not None:
            # Используем DatasetBlender для смешивания датасетов
            current_train_data, blend_ratio = dataset_blender.create_blended_dataset(
                current_epoch=epoch,
                random_seed=epoch * 12345
            )
            
            # Логируем метрики смешивания
            wandb.log({
                "dataset/blend_ratio": float(blend_ratio),
                "dataset/primary_examples": int(len(current_train_data) * (1 - blend_ratio)),
                "dataset/secondary_examples": int(len(current_train_data) * blend_ratio),
                "dataset/total_examples": int(len(current_train_data)),
                "dataset/epoch": int(epoch + 1)
            })
            
            # Выбираем правильный ZIP файл в зависимости от преобладающего датасета
            current_zip_path = secondary_zip_path if blend_ratio > 0.5 else primary_zip_path
        else:
            # Обычная логика без смешивания
            random_state = random.Random(epoch * 12345)  # Уникальный seed для каждой эпохи
            shuffled_indices = list(range(len(train_data)))
            random_state.shuffle(shuffled_indices)
            current_train_data = [train_data[i] for i in shuffled_indices]
            current_zip_path = primary_zip_path
            print(f"🔄 Эпоха {epoch+1}: детерминированно перемешано {len(current_train_data)} примеров")
        
        train_dataset = AudioTextDataset(current_train_data, tokenizer, feature_extractor, zip_path=current_zip_path)
        train_loader = DataLoader(
            train_dataset,
            batch_size=batch_size,
            shuffle=False,  # Данные уже детерминированно перемешаны
            collate_fn=collate_fn,
            num_workers=0,
            pin_memory=False
        )
    
    first_batch_logged = False
    accumulated_loss = 0.0

    progress_bar = tqdm(
        enumerate(train_loader), 
        total=len(train_loader),
        initial=batch_idx if is_resumed_epoch else 0,
        desc="Epoch " + str(epoch+1),
        leave=True,
        position=0,
        dynamic_ncols=True
    )

    for batch_idx, batch in progress_bar:
        try:
            # Простой и правильный расчет: global_step уже учитывает все предыдущие шаги
            # batch_idx из enumerate идет 0, 1, 2... для текущего DataLoader
            real_batch_number = global_step + batch_idx
            
            if not first_batch_logged:
                wandb.log({
                    "batch/audio_seq_len": int(batch['input_values'].shape[1]),
                    "batch/audio_batch_size": int(batch['input_values'].shape[0]),
                    "batch/text_seq_len": int(batch['input_ids'].shape[1]),
                    "batch/text_batch_size": int(batch['input_ids'].shape[0]),
                    "batch/grad_accum_steps": int(gradient_accumulation_steps)
                })
                first_batch_logged = True
                    
            current_global_step = real_batch_number
            
            outputs, _, norm_diagnostics = process_batch(
                batch, gemma_model, projector, wav2vec2, tokenizer, prefix_embeds, device, compression_rate_k
            )
            loss = outputs.loss
            del outputs  # Освобождаем память от логитов
            
            loss = loss / gradient_accumulation_steps
            accumulated_loss += loss.item()
            
        except torch.cuda.OutOfMemoryError:
            print_ephemeral(f"🔥 OOM шаг {current_global_step}, аудио: {batch['input_values'].shape}, текст: {batch['input_ids'].shape}")
            force_gpu_cleanup()
            wandb.log({
                "train/oom_skipped_batch": 1,
                "train/oom_audio_shape": str(batch['input_values'].shape),
                "train/oom_text_shape": str(batch['input_ids'].shape),
                "step": current_global_step
            })

        if not hasattr(projector, '_gpu_logged') and torch.cuda.is_available():
            projector._gpu_logged = True
            gpu_memory = torch.cuda.memory_allocated(device) / 1024**3
            gpu_memory_reserved = torch.cuda.memory_reserved(device) / 1024**3
            gpu_memory_max = torch.cuda.max_memory_allocated(device) / 1024**3
            
            # Numeric memory utilization classification  
            memory_util_numeric = 3.0 if gpu_memory > 20 else 2.0 if gpu_memory >= 8 else 1.0
            
            wandb.log({
                "gpu/memory_allocated_gb": float(gpu_memory),
                "gpu/memory_reserved_gb": float(gpu_memory_reserved),
                "gpu/memory_peak_gb": float(gpu_memory_max),
                "gpu/batch_size": int(batch_size),
                "gpu/memory_utilization_level": memory_util_numeric  # 1=low, 2=optimal, 3=high
            })
                
        scaler.scale(loss).backward()
        
        if (batch_idx + 1) % gradient_accumulation_steps == 0:
            scaler.unscale_(optimizer)
            
            # Вычисляем норму градиентов ДО clipping'а для диагностики
            grad_norm_before_clip = 0.0
            for param in projector.parameters():
                if param.grad is not None:
                    grad_norm_before_clip += param.grad.data.norm(2).item() ** 2
            grad_norm_before_clip = grad_norm_before_clip ** 0.5
            
            # Применяем clipping
            grad_norm = torch.nn.utils.clip_grad_norm_(projector.parameters(), max_grad_norm)
            
            # Определяем был ли clipping
            was_clipped = grad_norm_before_clip > max_grad_norm

            projector_l2_norm = projector.get_l2_norm()
            
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            optimizer.zero_grad()
            
            current_lr_list = scheduler.get_last_lr()  # Получаем весь список LR
            
            # --- Детальная диагностика памяти ---
            gpu_memory_used, gpu_memory_reserved, gpu_memory_total = get_gpu_memory_stats(device)
            
            memory_breakdown = {}
            memory_breakdown['projector'] = get_model_memory_footprint(projector)
            memory_breakdown['lora_adapter'] = get_model_memory_footprint(gemma_model, trainable_only=True)
            memory_breakdown['wav2vec2'] = get_model_memory_footprint(wav2vec2)

            trainable_params_count = sum(p.nelement() for p in projector.parameters() if p.requires_grad) + sum(p.nelement() for p in gemma_model.parameters() if p.requires_grad)
            memory_breakdown['optimizer_state'] = (trainable_params_count * 2) / 1024**2

            total_allocated_mb = gpu_memory_used * 1024 if gpu_memory_used is not None else 0
            model_related_mb = sum(memory_breakdown.values())
            memory_breakdown['activations_grads_misc'] = max(0, total_allocated_mb - model_related_mb)
            # --- Конец диагностики ---
            
            # Добавляем дополнительные метрики градиентов к memory_breakdown
            memory_breakdown['grad_norm_before_clip'] = grad_norm_before_clip
            memory_breakdown['grad_norm_after_clip'] = grad_norm.item()
            memory_breakdown['was_clipped'] = was_clipped
            memory_breakdown['clipping_ratio'] = grad_norm.item() / max(grad_norm_before_clip, 1e-8)
            
            # 🔍 Добавляем L2-нормы к диагностикам
            memory_breakdown.update(norm_diagnostics)
            
            # 📊 Вычисляем Update Ratio метрики для мониторинга эффективности обучения
            update_ratio_metrics = None
            if enable_update_ratio_monitoring and current_global_step % update_ratio_logging_frequency == 0:
                update_ratio_metrics = calculate_separate_update_ratios(projector, gemma_model)
                
                # ⚠️ Проверяем на микроскопические обновления
                if update_ratio_metrics:
                    for component, metrics in update_ratio_metrics.items():
                        if metrics['update_ratio'] < update_ratio_alert_threshold:
                            print(f"⚠️ Микроскопические обновления в {component}: {metrics['update_ratio']:.2e} < {update_ratio_alert_threshold:.1e}")
            
            logger.log_step(
                current_global_step, 
                accumulated_loss, 
                current_lr_list,  # Передаем весь список
                grad_norm.item(),
                projector_l2_norm,
                gpu_memory_used,
                gpu_memory_reserved,
                gpu_memory_total,
                memory_breakdown,
                update_ratio_metrics  # 📊 Новые метрики!
            )
            
            clip_info = f"[CLIPPED {grad_norm_before_clip:.2f}→{grad_norm.item():.2f}]" if was_clipped else ""
            
            # 📊 Добавляем информацию об update ratio (если включено в настройках)
            update_ratio_str = ""
            if show_update_ratio_in_progress and update_ratio_metrics:
                proj_info = ""
                lora_info = ""
                
                if 'projector' in update_ratio_metrics:
                    proj_ratio = update_ratio_metrics['projector']['update_ratio']
                    proj_emoji = update_ratio_metrics['projector']['update_ratio_emoji']
                    proj_info = f"Proj:{proj_emoji}{proj_ratio:.1e}"
                
                if 'lora' in update_ratio_metrics:
                    lora_ratio = update_ratio_metrics['lora']['update_ratio']
                    lora_emoji = update_ratio_metrics['lora']['update_ratio_emoji']
                    lora_info = f"LoRA:{lora_emoji}{lora_ratio:.1e}"
                
                if proj_info and lora_info:
                    update_ratio_str = f", UR=[{proj_info},{lora_info}]"
                elif proj_info:
                    update_ratio_str = f", UR=[{proj_info}]"
                elif lora_info:
                    update_ratio_str = f", UR=[{lora_info}]"
            
            metrics_str = f"Loss={accumulated_loss:.4f}, LR-Proj={current_lr_list[0]:.2e}, LR-LoRA={current_lr_list[1]:.2e}, GN={grad_norm.item():.2f}{clip_info}, L2={projector_l2_norm:.1f}{update_ratio_str}"
            mem_str = f"Mem(MB):Alloc={total_allocated_mb:.0f},Act={memory_breakdown['activations_grads_misc']:.0f}"
            progress_bar.set_postfix_str(f"{metrics_str} | {mem_str}")
            
            if current_global_step % 50 == 0:
                progress_bar.write(f"📊 Step {current_global_step}: {metrics_str} | {mem_str}")
            
            accumulated_loss = 0.0
        
        check_user_input()
        
        if current_global_step % save_latest_every_steps == 0:
            save_latest_checkpoint(current_global_step, epoch + 1, batch_idx)
            # Отладочная информация для понимания сохранения
            if current_global_step % 100 == 0:  # Каждые 100 шагов
                print_ephemeral(f"💾 Сохранен чекпоинт: epoch={epoch+1}, global_step={current_global_step}, batch_idx={batch_idx}")
        
        if current_global_step % save_every_steps == 0:
            if skip_validation:
                skip_validation = False
            else:
                # 🔬 Определяем, запускать ли исследовательский анализ
                run_analysis_now = (enable_embedding_analysis and 
                                  embedding_analyzer is not None and 
                                  current_global_step % analysis_frequency == 0)
                
                if run_analysis_now:
                    print(f"\n🔬 Шаг {current_global_step}: Запуск валидации с исследовательским анализом")
                
                # Используем расширенную функцию валидации с анализом
                val_metrics = evaluate_with_analysis(
                    gemma_model, projector, wav2vec2, val_loader, 
                    tokenizer, prefix_embeds, device, max_new_tokens, compression_rate_k,
                    beam_width, temperature, top_k, top_p, repetition_penalty,
                    analyzer=embedding_analyzer,
                    current_step=current_global_step,
                    run_analysis=run_analysis_now,
                    analysis_config=analysis_config
                )
            
                logger.log_validation(current_global_step, val_metrics)
                
                # Логируем дополнительные метрики анализа, если они есть
                if 'analysis' in val_metrics and run_analysis_now:
                    wandb.log({
                        "analysis/num_batches_analyzed": len(val_metrics['analysis']),
                        "analysis/analysis_step": current_global_step,
                        "step": current_global_step
                    })
                
                is_best = val_metrics['loss'] < best_val_loss
                if is_best:
                    best_val_loss = val_metrics['loss']
                    print_ephemeral(f"🏆 Новый лучший результат! Loss: {best_val_loss:.4f}")
                
                save_checkpoint(current_global_step, epoch + 1, batch_idx, is_best)
                
                # Очищаем память (включая результаты анализа)
                if 'analysis' in val_metrics:
                    del val_metrics['analysis']
                del val_metrics
                torch.cuda.empty_cache()
                
                projector.train()
    
    if is_resumed_epoch:
        resume_training = False
        print_vanishing("✅ Эпоха " + str(epoch+1) + " завершена, переходим к обычному режиму")

In [ ]:
print_ephemeral("🎉 Обучение завершено! Финальная валидация...")

full_val_dataset = AudioTextDataset(val_data, tokenizer, feature_extractor, zip_path=zip_path)
full_val_loader = DataLoader(full_val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

final_val_metrics = evaluate_with_metrics(
    gemma_model, projector, wav2vec2, full_val_loader, 
    tokenizer, prefix_embeds, device, max_new_tokens, compression_rate_k,
    beam_width, temperature, top_k, top_p, repetition_penalty
)

# Финальные результаты показываем эфемерно
print_ephemeral(f"📊 Final: Loss={final_val_metrics['loss']:.4f} PPL={final_val_metrics['perplexity']:.2f} WER={final_val_metrics['wer']:.3f}")

logger.log_validation(global_step, final_val_metrics)

del final_val_metrics, full_val_dataset, full_val_loader
torch.cuda.empty_cache()

logger.save_logs()

final_model_path = os.path.join(checkpoint_dir, "final_projector.pt")
torch.save(projector.state_dict(), final_model_path)
print_ephemeral(f"🏆 Модель сохранена: {final_model_path}")

wandb.finish()
torch.cuda.empty_cache()
gc.collect()
print_ephemeral("✅ Завершено")

In [ ]:
print_ephemeral("🎉 Обучение завершено! Финальная валидация...")

full_val_dataset = AudioTextDataset(val_data, tokenizer, feature_extractor, zip_path=zip_path)
full_val_loader = DataLoader(full_val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

final_val_metrics = evaluate_with_metrics(
    gemma_model, projector, wav2vec2, full_val_loader, 
    tokenizer, prefix_embeds, device, max_new_tokens, compression_rate_k,
    beam_width, temperature, top_k, top_p, repetition_penalty
)

# Финальные результаты показываем эфемерно
print_ephemeral(f"📊 Final: Loss={final_val_metrics['loss']:.4f} PPL={final_val_metrics['perplexity']:.2f} WER={final_val_metrics['wer']:.3f}")

logger.log_validation(global_step, final_val_metrics)

del final_val_metrics, full_val_dataset, full_val_loader
torch.cuda.empty_cache()

logger.save_logs()

final_model_path = os.path.join(checkpoint_dir, "final_projector.pt")
torch.save(projector.state_dict(), final_model_path)
print_ephemeral(f"🏆 Модель сохранена: {final_model_path}")

wandb.finish()
torch.cuda.empty_cache()
gc.collect()
print_ephemeral("✅ Завершено")

In [ ]:
# 🚀 ОБНОВЛЕННЫЙ КОД ДЛЯ БОРЬБЫ СО СТАГНАЦИЕЙ ОБУЧЕНИЯ

## ✅ Все новые изменения для выхода из локальных минимумов:

### 1. **🚫 MSE Loss убран из-за фундаментальных проблем**
- ❌ **Проблема выравнивания**: Невозможно корректно сопоставить непрерывный аудио-поток с дискретными токенами
- ❌ **Игнорирование LLM**: MSE не учитывает внутреннюю логику замороженной Gemma 
- ❌ **Косвенная оптимизация**: L2-близость эмбеддингов не гарантирует высокую вероятность правильного токена
- ✅ **Решение**: Используем только Cross-Entropy loss для end-to-end обучения через LLM

### 2. **🔄 CosineAnnealingWarmRestarts для выхода из локальных минимумов**
- ✅ **Заменен OneCycleLR**: Теперь используется CosineAnnealingWarmRestarts
- ✅ **Частые рестарты**: T_0=250 шагов (~каждые 15 минут обучения)
- ✅ **Константный период**: T_mult=1 (период не увеличивается)
- ✅ **Минимальный LR**: 1e-6 перед каждым рестартом
- ✅ **Выход из минимумов**: Регулярные скачки LR помогают выйти из локальных минимумов

### 3. **🚀 Увеличенный Learning Rate в 3 раза**
- ✅ **С 1e-3 до 3e-3**: Агрессивный подход для преодоления стагнации
- ✅ **Больше свободы**: Проектор получает больше энергии для изменений
- ✅ **Сочетание с рестартами**: LR периодически сбрасывается, предотвращая расхождение

### 4. **📊 Очищенное логирование и мониторинг**
- ✅ **Убран MSE Loss**: Больше не отслеживается избыточный MSE loss
- ✅ **Все метрики**: Projector L2 norm, weight update ratio, gradient norm
- ✅ **Scheduler type**: Flexibile выбор между "onecycle" и "cosine_restarts"
- ✅ **Restart параметры**: T_0, T_mult, eta_min в конфигурации

### 5. **🔧 Централизованные гиперпараметры для scheduler**
- ✅ **scheduler_type**: Легкое переключение между подходами
- ✅ **cosine_restart_period**: Настройка периода рестарта
- ✅ **cosine_restart_mult**: Контроль роста периода
- ✅ **cosine_eta_min**: Минимальный LR для рестартов
- 🚫 **mse_loss_weight**: Убран вместе с MSE loss

## 🎯 Механизм борьбы со стагнацией:

**ПРОБЛЕМА**: Train loss быстро падает с 6-7 до ~3.5, затем стагнирует  
**ПРИЧИНА**: Маленький проектор (7M параметров) быстро находит локальный минимум  

**РЕШЕНИЯ**:
1. **🚫 Убран MSE Loss**: Избегаем проблем с выравниванием, доверяем LLM feedback
2. **🔄 Warm Restarts**: Периодические "толчки" LR для выхода из локальных минимумов  
3. **🚀 3x Learning Rate**: Больше энергии для изменения весов
4. **📊 Мониторинг**: Отслеживание всех ключевых метрик для диагностики

## 📈 Ожидаемые улучшения:

1. **📉 Преодоление стагнации**: Loss должен продолжать падать после ~3.5
2. **🎯 Лучший WER**: Чистый end-to-end сигнал от LLM без искажений от MSE
3. **🔄 Стабильное обучение**: Рестарты предотвращают застревание
4. **⚡ Фокус на Cross-Entropy**: Проектор учится "говорить" на языке LLM

## 🚀 Готово к запуску с агрессивными настройками против стагнации!

In [ ]:
# 🔬 ПРИМЕР ИСПОЛЬЗОВАНИЯ АНАЛИЗАТОРА (для тестирования)
# Этот код можно запустить после обучения для анализа конкретных примеров

def test_embedding_analyzer(analyzer, val_loader, prefix_embeds, compression_rate_k, device, sample_name="test"):
    """
    Тестовая функция для проверки работы анализатора
    
    Запускает анализ на одном батче из валидационного набора
    """
    if analyzer is None:
        print("❌ Анализатор не инициализирован")
        return
    
    print(f"🔬 Тестирование анализатора на примере '{sample_name}'...")
    
    # Берем первый батч для анализа
    for batch in val_loader:
        try:
            # Запускаем комплексный анализ
            results = analyzer.run_comprehensive_analysis(
                batch=batch,
                compression_rate_k=compression_rate_k,
                prefix_embeds=prefix_embeds,
                sample_name=sample_name
            )
            
            # Выводим краткий отчет
            print(f"\n📊 Результаты анализа для '{sample_name}':")
            print(f"   🎯 Diagonal Alignment: {results['alignment_metrics']['diagonal_alignment_score']:.4f}")
            print(f"   📈 Max Alignment: {results['alignment_metrics']['max_alignment_score']:.4f}")
            print(f"   🔄 Attention Entropy: {results['alignment_metrics']['attention_entropy']:.4f}")
            print(f"   📊 Similarity Mean: {results['similarity_stats']['mean_similarity']:.4f}")
            print(f"   📏 Similarity Std: {results['similarity_stats']['std_similarity']:.4f}")
            
            # Показываем примеры ближайших токенов
            if 'nearest_tokens' in results and len(results['nearest_tokens']) > 0:
                print(f"\n🔍 Ближайшие токены для первых временных шагов:")
                for i, timestep in enumerate(results['nearest_tokens'][:3]):
                    top_token = timestep['nearest_tokens'][0]
                    print(f"   Шаг {timestep['timestep']}: '{top_token['token_text']}' (similarity: {top_token['similarity']:.4f})")
            
            print(f"✅ Анализ завершен! Графики сохранены в папке: {analyzer.save_dir}")
            return results
            
        except Exception as e:
            print(f"⚠️ Ошибка при тестировании анализатора: {e}")
            return None
        
        break  # Анализируем только первый батч

# Инструкции по использованию
print("🔬 ИНСТРУКЦИИ ПО ИСПОЛЬЗОВАНИЮ АНАЛИЗАТОРА:")
print("1. После завершения обучения запустите:")
print("   test_embedding_analyzer(embedding_analyzer, val_loader, prefix_embeds, compression_rate_k, device)")
print("2. Для анализа конкретного батча во время обучения анализатор автоматически запускается")
print(f"   каждые {analysis_frequency} шагов (если enable_embedding_analysis=True)")
print("3. Все графики сохраняются в папку:", analysis_save_dir)
print("4. Метрики анализа логируются в wandb в секции 'analysis/'")

print(f"\n🔬 Текущая конфигурация анализа:")
print(f"   📊 Анализ включен: {'✅' if enable_embedding_analysis else '❌'}")
print(f"   🕐 Частота анализа: каждые {analysis_frequency} шагов")
print(f"   📁 Папка сохранения: {analysis_save_dir}")
print(f"   🎯 t-SNE анализ: {'✅' if run_tsne_analysis else '❌'}")
print(f"   🔥 Матрицы сходства: {'✅' if run_similarity_heatmap else '❌'}")
print(f"   🔍 Поиск токенов: {'✅' if run_nearest_tokens_analysis else '❌'}")

print(f"\n📊 Конфигурация мониторинга Update Ratio:")
print(f"   🔄 Мониторинг включен: {'✅' if enable_update_ratio_monitoring else '❌'}")
print(f"   🕐 Частота логирования: каждые {update_ratio_logging_frequency} шагов")
print(f"   📱 Показ в прогресс-баре: {'✅' if show_update_ratio_in_progress else '❌'}")
print(f"   ⚠️ Порог предупреждений: {update_ratio_alert_threshold:.1e}")
print(f"   📈 Оптимальный диапазон: 1e-4 - 1e-3")
print(f"   🟢 Что означают эмодзи:")
print(f"      🔴 < 1e-5: микроскопические обновления")
print(f"      🟡 1e-5 - 1e-4: маленькие, но допустимые")
print(f"      🟢 1e-4 - 1e-3: оптимальный диапазон")
print(f"      🟠 > 1e-3: слишком большие обновления")


In [ ]:
# 📊 Демонстрация Update Ratio мониторинга
print("📊 Update Ratio мониторинг настроен и готов к использованию!")
print(f"🔄 Статус: {'✅ Включен' if enable_update_ratio_monitoring else '❌ Отключен'}")
print(f"📱 Прогресс-бар: {'✅ Показывать' if show_update_ratio_in_progress else '❌ Скрыть'}")

# Демонстрация функции (пример)
if enable_update_ratio_monitoring:
    print("\n🧪 Пример вычисления Update Ratio:")
    # Демонстрационные значения
    demo_grad_norm = 2.5e-4
    demo_weight_norm = 1.2
    demo_metrics = calculate_update_ratio_metrics(demo_grad_norm, demo_weight_norm)
    
    print(f"   Grad norm: {demo_metrics['grad_norm']:.2e}")
    print(f"   Weight norm: {demo_metrics['weight_norm']:.2f}")
    print(f"   Update ratio: {demo_metrics['update_ratio']:.2e}")
    print(f"   Статус: {demo_metrics['update_ratio_status']} {demo_metrics['update_ratio_emoji']}")
    print(f"   Числовой статус: {demo_metrics['update_ratio_numeric']}")

print("\n🚀 Все готово для запуска обучения с полным мониторингом Update Ratio!")


In [ ]:
# 📊 МОНИТОРИНГ UPDATE RATIO - КОНТРОЛЬ ЭФФЕКТИВНОСТИ ОБУЧЕНИЯ

## 🎯 Что такое Update Ratio?

**Update Ratio** = `grad_norm / (weight_norm + 1e-8)` - показывает относительную "силу" обновления весов.

### 📈 Интерпретация значений:

- **🔴 < 1e-5**: Микроскопические обновления - возможно, слишком низкий learning rate
- **🟡 1e-5 - 1e-4**: Маленькие, но допустимые обновления  
- **🟢 1e-4 - 1e-3**: **ОПТИМАЛЬНЫЙ ДИАПАЗОН** - рабочие значения для большинства задач
- **🟠 > 1e-3**: Слишком большие обновления - возможно, слишком высокий learning rate

## 🔬 Что мониторится:

1. **Projector Update Ratio**: Отношение градиентов к весам в аудио-проекторе
2. **LoRA Update Ratio**: Отношение градиентов к весам в LoRA адаптерах Gemma-3
3. **Grad/Weight Norms**: Отдельные нормы градиентов и весов для диагностики

## 📊 Где смотреть результаты:

### 1. **В прогресс-баре (если включено)**
```
Loss=2.1234, LR-Proj=3.0e-3, LR-LoRA=1.0e-3, GN=1.23, L2=45.6, UR=[Proj:🟢2.3e-4,LoRA:🟡8.9e-5]
```

### 2. **В Weights & Biases**
- `update_ratio/projector_ratio` - update ratio проектора
- `update_ratio/projector_status_numeric` - числовой статус (1-4)
- `update_ratio/projector_grad_norm` - норма градиентов проектора
- `update_ratio/projector_weight_norm` - норма весов проектора
- `update_ratio/lora_ratio` - update ratio LoRA
- `update_ratio/lora_status_numeric` - числовой статус LoRA
- `update_ratio/lora_grad_norm` - норма градиентов LoRA
- `update_ratio/lora_weight_norm` - норма весов LoRA

### 3. **В консоли (предупреждения)**
```
⚠️ Микроскопические обновления в projector: 8.45e-06 < 1.0e-05
⚠️ Микроскопические обновления в lora: 3.21e-06 < 1.0e-05
```

## 🔧 Настройки конфигурации:

```python
# 📊 Параметры мониторинга Update Ratio
enable_update_ratio_monitoring = True  # Включить/отключить мониторинг
update_ratio_logging_frequency = 1     # Каждые N шагов (1 = каждый шаг)
show_update_ratio_in_progress = True   # Показывать в прогресс-баре с эмодзи
update_ratio_alert_threshold = 1e-5    # Порог для предупреждений
```

## 🚨 Диагностика проблем:

### **Проблема: Микроскопические обновления (🔴)**
**Симптомы**: Update ratio < 1e-5, обучение стагнирует
**Решения**:
- Увеличить learning rate
- Проверить gradient clipping (возможно, слишком агрессивный)
- Проверить weight decay (возможно, слишком высокий)

### **Проблема: Слишком большие обновления (🟠)**
**Симптомы**: Update ratio > 1e-3, loss взрывается или осциллирует
**Решения**:
- Уменьшить learning rate
- Усилить gradient clipping
- Проверить weight decay

### **Проблема: Дисбаланс между компонентами**
**Симптомы**: Projector🟢, LoRA🔴 (или наоборот)
**Решения**:
- Настроить отдельные learning rates
- Проверить, не заморожены ли параметры одного из компонентов

## ✅ Пример здорового обучения:

```
📊 Step 1500: Loss=2.345, UR=[Proj:🟢3.2e-4,LoRA:🟢1.8e-4] | Mem(MB):Alloc=8234,Act=1024
📊 Step 1550: Loss=2.298, UR=[Proj:🟢2.9e-4,LoRA:🟢2.1e-4] | Mem(MB):Alloc=8156,Act=987
```

**Интерпретация**: Оба компонента работают в оптимальном диапазоне, обучение здоровое!
